In [1]:
cd ../py_scripts/

/Users/r0b00sj/Documents/Capstone/Capstone/py_scripts


In [64]:
import pandas as pd
from gensim.models.coherencemodel import CoherenceModel
from utils_streamers import DirFileMgr, CorpStreamer, BOWCorpStreamer
from gensim import corpora
from gensim.models.ldamodel import LdaModel
import json
import pyLDAvis
import pyLDAvis.gensim

In [49]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
rootLogger = logging.getLogger()
rootLogger.setLevel(logging.INFO)

pd.options.display.max_colwidth = 500

In [4]:
dr_id_str = "dr_500k"
dr_fps = DirFileMgr(dr_id_str)
dr_fps.create_all_dr_fps(new_setup='N')

Which data set will be used? Enter either '5000' or 'full': full
corpus lst fp is assigned as  ../../outputs-git_ignored/dr_500k/dr_500k_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/dr_500k/dr_500k.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/dr_500k/dr_500k_json.txt
dimensional reduction run parameters fp is assigned as  ../../outputs-git_ignored/dr_500k/dr_500k_dr_run_params.txt


In [6]:
d = corpora.Dictionary.load(dr_fps.dictionary_fp)
c = CorpStreamer(d, dr_fps.corp_lst_fp, inc_title='Y')
corp_lst_stream = CorpStreamer(dictionary=d, corp_lst_fp=dr_fps.corp_lst_fp)

# bow_c = BOWCorpStreamer(d, dr_fps.corp_lst_fp, inc_title='Y')
# dr_fps.add_fp('gensim_corpus')

#long running time - execute only for a new dimensional reduction run
# if 0 == 1:
#     corpora.MmCorpus.serialize(dr_fps.gensim_corp_fp, bow_c)
    
# #whether the serialized corpus has just been already been created and saved (above), 
# # or was created and saved earlier (as for demo) we load it with this expression:
# serial_corp = corpora.MmCorpus(dr_fps.gensim_corp_fp)



2018-03-27 12:13:35,707 : INFO : loading Dictionary object from ../../outputs-git_ignored/dr_500k/dr_500k.dict
2018-03-27 12:13:35,884 : INFO : loaded ../../outputs-git_ignored/dr_500k/dr_500k.dict
2018-03-27 12:13:35,892 : INFO : loaded corpus index from ../../outputs-git_ignored/dr_500k/dr_500k_corpus.mm.index
2018-03-27 12:13:35,894 : INFO : initializing cython corpus reader from ../../outputs-git_ignored/dr_500k/dr_500k_corpus.mm
2018-03-27 12:13:35,899 : INFO : accepted corpus with 28492 documents, 262826 features, 90336941 non-zero entries


corpus fp is assigned as  ../../outputs-git_ignored/dr_500k/dr_500k_corpus.mm


In [65]:
json

<module 'json' from '/anaconda3/lib/python3.6/json/__init__.py'>

In [61]:
def analyze_model_coherence(mod_id_str, corp_lst_stream):   
    """
    :param {str} mod_id_str:
        The identifying string for the model to be analyzed
    :param {CorpStreamer} corp_lst_str:
        Streaming corpus object
    
    Loads and creates all objects to build a coherence model and associate it with the model topic terms.
    """
    #set up filepaths for this model for this task
    fps = DirFileMgr(mod_id_str)
    fps.create_all_modeling_fps(mod_id_str)
    fps.add_fp('pyLDAvis')
    fps.add_fp('coherence_model')
    fps.add_fp('mod_run_params')

    #load the trained LDA model
    lda = LdaModel.load(fps.model_fp)
    
    #load model run parameters
    with open(fps.mod_run_params) as run_params_f:
        run_params = json.load(run_params_f)
    
    #create and save pyLDAvis for visual inspection
    lda_vis_serialized = pyLDAvis.gensim.prepare(lda, serial_corp, d)
    pyLDAvis.save_html(lda_vis_serialized, fps.pyldavis_fp)

    #train and save coherence model
    cm_cv = CoherenceModel(model=lda, \
                        texts=corp_lst_stream, \
                        coherence='c_v')
    cm_cv.save(fps.coherence_model_fp + 'cv')

    #get coherence scores - for model overall, and for individual topics
    cv_coherence = cm_cv.get_coherence()
    cv_topic_coherence = cm_cv.get_coherence_per_topic()

    #create a dataframe that holds the relevant info for analyzing the model
    df = pd.DataFrame(data=lda.show_topics(num_topics=lda.num_topics), columns=['topic_num', 'topic_terms'])
    df.set_index(keys='topic_num', inplace=True)
    df['cv_coherence'] = cv_topic_coherence
    df.sort_values(by=['cv_coherence'], ascending=False, inplace=True)

    return df, cv_coherence, cv_topic_coherence, run_params

In [63]:
mod_1_df, mod_1_cv_coherence, mod_1_cv_topic_coherence, mod_1_run_params =\
    analyze_model_coherence('gpu_mod_run_1', corp_lst_stream)
    
print(mod_1_cv_coherence, mod_1_run_params)
mod_1_df

2018-03-28 12:21:07,000 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model
2018-03-28 12:21:07,020 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model.expElogbeta.npy with mmap=None
2018-03-28 12:21:07,120 : INFO : setting ignored attribute dispatcher to None
2018-03-28 12:21:07,121 : INFO : setting ignored attribute state to None
2018-03-28 12:21:07,123 : INFO : setting ignored attribute id2word to None
2018-03-28 12:21:07,124 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model
2018-03-28 12:21:07,126 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model, assuming np.float64
2018-03-28 12:21:07,128 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model.state
2018-03-28 12:21:07,136 : INFO : loading sstats from ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model.sta

corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_mod_run_params.txt


2018-03-28 12:21:07,231 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model.state
2018-03-28 12:21:07,232 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-03-28 12:27:55,451 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_coherence_model.cv, separately None
2018-03-28 12:27:55,453 : INFO : storing np array 'sstats' to ../../outputs-git_ignored/gpu_mod_run_1/gpu_mod_run_1_coherence_model.cv._model.state.sstats.npy
2018-03-28 12:27:55,522 : INFO : storing np array 'expElogbeta' to ../.

2018-03-28 12:30:35,987 : INFO : 62 batches submitted to accumulate stats from 3968 documents (35703571 virtual)
2018-03-28 12:30:43,768 : INFO : 63 batches submitted to accumulate stats from 4032 documents (36210981 virtual)
2018-03-28 12:30:44,393 : INFO : 64 batches submitted to accumulate stats from 4096 documents (36663722 virtual)
2018-03-28 12:30:45,207 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37202399 virtual)
2018-03-28 12:30:46,225 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37560385 virtual)
2018-03-28 12:30:48,431 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38196793 virtual)
2018-03-28 12:30:50,972 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38650823 virtual)
2018-03-28 12:30:54,827 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39098416 virtual)
2018-03-28 12:30:59,162 : INFO : 70 batches submitted to accumulate stats from 4480 documents (3

2018-03-28 12:33:44,582 : INFO : 135 batches submitted to accumulate stats from 8640 documents (76260036 virtual)
2018-03-28 12:33:45,691 : INFO : 136 batches submitted to accumulate stats from 8704 documents (76750590 virtual)
2018-03-28 12:33:47,757 : INFO : 137 batches submitted to accumulate stats from 8768 documents (77411895 virtual)
2018-03-28 12:33:52,885 : INFO : 138 batches submitted to accumulate stats from 8832 documents (77996318 virtual)
2018-03-28 12:33:57,071 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78525533 virtual)
2018-03-28 12:33:59,970 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79115828 virtual)
2018-03-28 12:34:00,812 : INFO : 141 batches submitted to accumulate stats from 9024 documents (79690300 virtual)
2018-03-28 12:34:02,919 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80250005 virtual)
2018-03-28 12:34:03,543 : INFO : 143 batches submitted to accumulate stats from 9152 doc

...

2018-03-28 12:39:26,935 : INFO : 278 batches submitted to accumulate stats from 17792 documents (146784834 virtual)
2018-03-28 12:39:29,007 : INFO : 279 batches submitted to accumulate stats from 17856 documents (147232939 virtual)
2018-03-28 12:39:30,409 : INFO : 280 batches submitted to accumulate stats from 17920 documents (147659090 virtual)
2018-03-28 12:39:31,321 : INFO : 281 batches submitted to accumulate stats from 17984 documents (148178356 virtual)
2018-03-28 12:39:33,539 : INFO : 282 batches submitted to accumulate stats from 18048 documents (148711007 virtual)
2018-03-28 12:39:34,057 : INFO : 283 batches submitted to accumulate stats from 18112 documents (149103597 virtual)
2018-03-28 12:39:35,612 : INFO : 284 batches submitted to accumulate stats from 18176 documents (149423813 virtual)
2018-03-28 12:39:42,279 : INFO : 285 batches submitted to accumulate stats from 18240 documents (149836253 virtual)
2018-03-28 12:39:43,800 : INFO : 286 batches submitted to accumulate sta

2018-03-28 12:42:18,087 : INFO : 349 batches submitted to accumulate stats from 22336 documents (185364673 virtual)
2018-03-28 12:42:21,868 : INFO : 350 batches submitted to accumulate stats from 22400 documents (185891121 virtual)
2018-03-28 12:42:25,102 : INFO : 351 batches submitted to accumulate stats from 22464 documents (186313818 virtual)
2018-03-28 12:42:25,375 : INFO : 352 batches submitted to accumulate stats from 22528 documents (186767594 virtual)
2018-03-28 12:42:27,708 : INFO : 353 batches submitted to accumulate stats from 22592 documents (187256362 virtual)
2018-03-28 12:42:31,999 : INFO : 354 batches submitted to accumulate stats from 22656 documents (187708641 virtual)
2018-03-28 12:42:38,034 : INFO : 355 batches submitted to accumulate stats from 22720 documents (188310645 virtual)
2018-03-28 12:42:39,372 : INFO : 356 batches submitted to accumulate stats from 22784 documents (188856412 virtual)
2018-03-28 12:42:44,758 : INFO : 357 batches submitted to accumulate sta

2018-03-28 12:45:39,243 : INFO : 420 batches submitted to accumulate stats from 26880 documents (228405381 virtual)
2018-03-28 12:45:42,654 : INFO : 421 batches submitted to accumulate stats from 26944 documents (228875888 virtual)
2018-03-28 12:45:43,303 : INFO : 422 batches submitted to accumulate stats from 27008 documents (229520266 virtual)
2018-03-28 12:45:43,801 : INFO : 423 batches submitted to accumulate stats from 27072 documents (230260048 virtual)
2018-03-28 12:45:52,081 : INFO : 424 batches submitted to accumulate stats from 27136 documents (230868972 virtual)
2018-03-28 12:45:55,944 : INFO : 425 batches submitted to accumulate stats from 27200 documents (231490388 virtual)
2018-03-28 12:45:56,424 : INFO : 426 batches submitted to accumulate stats from 27264 documents (232133623 virtual)
2018-03-28 12:45:59,172 : INFO : 427 batches submitted to accumulate stats from 27328 documents (232778899 virtual)
2018-03-28 12:46:01,851 : INFO : 428 batches submitted to accumulate sta

0.4412697789628097 {'num_topics': 50, 'cores': 2, 'chunksize': 2000, 'passes': 10, 'batch': False, 'alpha': 'symmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01, 'per_word_topic': 0.01, 'update_every': 0}


,topic_terms,cv_coherence
topic_num,,
0,"0.004*""boil"" + 0.003*""salt"" + 0.003*""pound"" + 0.003*""butter"" + 0.003*""sugar"" + 0.003*""milk"" + 0.003*""pour"" + 0.002*""pint"" + 0.002*""cream"" + 0.002*""mix""",0.916451
38,"0.012*""thy"" + 0.011*""thou"" + 0.007*""unto"" + 0.005*""hath"" + 0.004*""ye"" + 0.004*""thee"" + 0.003*""christ"" + 0.002*""jesus"" + 0.002*""divine"" + 0.002*""hast""",0.803529
2,"0.008*""km"" + 0.005*""billion"" + 0.005*""na"" + 0.004*""million"" + 0.004*""population"" + 0.004*""military"" + 0.004*""national"" + 0.003*""islands"" + 0.003*""total"" + 0.003*""december""",0.742322
44,"0.005*""jesus"" + 0.003*""unto"" + 0.003*""christ"" + 0.003*""paul"" + 0.002*""thy"" + 0.002*""jewish"" + 0.002*""jews"" + 0.002*""ye"" + 0.001*""gospel"" + 0.001*""thou""",0.737823
45,"0.006*""dat"" + 0.006*""en"" + 0.005*""dey"" + 0.003*""ter"" + 0.003*""la"" + 0.003*""el"" + 0.002*""wid"" + 0.002*""que"" + 0.002*""git"" + 0.002*""wuz""",0.681790
17,"0.005*""british"" + 0.004*""german"" + 0.003*""national"" + 0.003*""president"" + 0.003*""military"" + 0.003*""troops"" + 0.003*""political"" + 0.002*""congress"" + 0.002*""officers"" + 0.002*""committee""",0.629701
14,"0.020*""sir"" + 0.002*""duke"" + 0.002*""earl"" + 0.002*""thou"" + 0.002*""knight"" + 0.002*""william"" + 0.002*""thy"" + 0.002*""queen"" + 0.002*""ye"" + 0.002*""henry""",0.620734
6,"0.010*""thou"" + 0.008*""thy"" + 0.004*""thee"" + 0.003*""ye"" + 0.002*""hath"" + 0.001*""hast"" + 0.001*""ere"" + 0.001*""prince"" + 0.001*""unto"" + 0.001*""bird""",0.618620
28,"0.002*""divine"" + 0.002*""spiritual"" + 0.002*""mental"" + 0.002*""science"" + 0.002*""relation"" + 0.002*""principle"" + 0.002*""conditions"" + 0.001*""principles"" + 0.001*""theory"" + 0.001*""hence""",0.601337


In [66]:
mod_2_df, mod_2_cv_coherence, mod_2_cv_topic_coherence, mod_2_run_params =\
    analyze_model_coherence('gpu_mod_run_2', corp_lst_stream)
    
print(mod_2_cv_coherence, mod_2_run_params)
mod_2_df

2018-03-28 12:53:37,421 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model
2018-03-28 12:53:37,440 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model.expElogbeta.npy with mmap=None
2018-03-28 12:53:37,538 : INFO : setting ignored attribute dispatcher to None
2018-03-28 12:53:37,539 : INFO : setting ignored attribute state to None
2018-03-28 12:53:37,540 : INFO : setting ignored attribute id2word to None
2018-03-28 12:53:37,541 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model
2018-03-28 12:53:37,542 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model, assuming np.float64
2018-03-28 12:53:37,544 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model.state
2018-03-28 12:53:37,558 : INFO : loading sstats from ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model.sta

corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_mod_run_params.txt


2018-03-28 12:53:37,649 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model.state
2018-03-28 12:53:37,650 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-03-28 13:00:35,883 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_coherence_model.cv, separately None
2018-03-28 13:00:35,885 : INFO : storing np array 'sstats' to ../../outputs-git_ignored/gpu_mod_run_2/gpu_mod_run_2_coherence_model.cv._model.state.sstats.npy
2018-03-28 13:00:35,948 : INFO : storing np array 'expElogbeta' to ../.

2018-03-28 13:03:02,783 : INFO : 62 batches submitted to accumulate stats from 3968 documents (35703571 virtual)
2018-03-28 13:03:09,273 : INFO : 63 batches submitted to accumulate stats from 4032 documents (36210981 virtual)
2018-03-28 13:03:10,596 : INFO : 64 batches submitted to accumulate stats from 4096 documents (36663722 virtual)
2018-03-28 13:03:11,251 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37202399 virtual)
2018-03-28 13:03:12,419 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37560385 virtual)
2018-03-28 13:03:13,928 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38196793 virtual)
2018-03-28 13:03:17,030 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38650823 virtual)
2018-03-28 13:03:18,373 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39098416 virtual)
2018-03-28 13:03:24,397 : INFO : 70 batches submitted to accumulate stats from 4480 documents (3

2018-03-28 13:06:05,078 : INFO : 135 batches submitted to accumulate stats from 8640 documents (76260036 virtual)
2018-03-28 13:06:06,115 : INFO : 136 batches submitted to accumulate stats from 8704 documents (76750590 virtual)
2018-03-28 13:06:08,808 : INFO : 137 batches submitted to accumulate stats from 8768 documents (77411895 virtual)
2018-03-28 13:06:13,563 : INFO : 138 batches submitted to accumulate stats from 8832 documents (77996318 virtual)
2018-03-28 13:06:16,164 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78525533 virtual)
2018-03-28 13:06:20,094 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79115828 virtual)
2018-03-28 13:06:22,277 : INFO : 141 batches submitted to accumulate stats from 9024 documents (79690300 virtual)
2018-03-28 13:06:22,637 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80250005 virtual)
2018-03-28 13:06:23,732 : INFO : 143 batches submitted to accumulate stats from 9152 doc

...

2018-03-28 13:11:28,386 : INFO : 278 batches submitted to accumulate stats from 17792 documents (146784799 virtual)
2018-03-28 13:11:30,996 : INFO : 279 batches submitted to accumulate stats from 17856 documents (147232904 virtual)
2018-03-28 13:11:32,019 : INFO : 280 batches submitted to accumulate stats from 17920 documents (147659055 virtual)
2018-03-28 13:11:32,868 : INFO : 281 batches submitted to accumulate stats from 17984 documents (148178321 virtual)
2018-03-28 13:11:34,874 : INFO : 282 batches submitted to accumulate stats from 18048 documents (148710972 virtual)
2018-03-28 13:11:36,009 : INFO : 283 batches submitted to accumulate stats from 18112 documents (149103562 virtual)
2018-03-28 13:11:38,160 : INFO : 284 batches submitted to accumulate stats from 18176 documents (149423778 virtual)
2018-03-28 13:11:43,303 : INFO : 285 batches submitted to accumulate stats from 18240 documents (149836218 virtual)
2018-03-28 13:11:45,205 : INFO : 286 batches submitted to accumulate sta

2018-03-28 13:14:10,324 : INFO : 349 batches submitted to accumulate stats from 22336 documents (185376177 virtual)
2018-03-28 13:14:13,779 : INFO : 350 batches submitted to accumulate stats from 22400 documents (185898641 virtual)
2018-03-28 13:14:16,622 : INFO : 351 batches submitted to accumulate stats from 22464 documents (186316095 virtual)
2018-03-28 13:14:17,298 : INFO : 352 batches submitted to accumulate stats from 22528 documents (186769649 virtual)
2018-03-28 13:14:19,259 : INFO : 353 batches submitted to accumulate stats from 22592 documents (187257660 virtual)
2018-03-28 13:14:23,740 : INFO : 354 batches submitted to accumulate stats from 22656 documents (187725695 virtual)
2018-03-28 13:14:29,137 : INFO : 355 batches submitted to accumulate stats from 22720 documents (188316870 virtual)
2018-03-28 13:14:30,577 : INFO : 356 batches submitted to accumulate stats from 22784 documents (188857737 virtual)
2018-03-28 13:14:34,927 : INFO : 357 batches submitted to accumulate sta

2018-03-28 13:17:20,336 : INFO : 420 batches submitted to accumulate stats from 26880 documents (228415227 virtual)
2018-03-28 13:17:23,703 : INFO : 421 batches submitted to accumulate stats from 26944 documents (228880671 virtual)
2018-03-28 13:17:24,803 : INFO : 422 batches submitted to accumulate stats from 27008 documents (229525568 virtual)
2018-03-28 13:17:25,313 : INFO : 423 batches submitted to accumulate stats from 27072 documents (230270271 virtual)
2018-03-28 13:17:32,189 : INFO : 424 batches submitted to accumulate stats from 27136 documents (230869983 virtual)
2018-03-28 13:17:34,529 : INFO : 425 batches submitted to accumulate stats from 27200 documents (231514702 virtual)
2018-03-28 13:17:36,691 : INFO : 426 batches submitted to accumulate stats from 27264 documents (232138940 virtual)
2018-03-28 13:17:38,601 : INFO : 427 batches submitted to accumulate stats from 27328 documents (232782381 virtual)
2018-03-28 13:17:42,082 : INFO : 428 batches submitted to accumulate sta

0.4652977169618066 {'num_topics': 50, 'cores': 41, 'chunksize': 2000, 'passes': 5, 'batch': False, 'alpha': 'symmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01, 'per_word_topic': 0.01, 'update_every': 0, 'dim_reduction_run_used': 'dr_500k'}


,topic_terms,cv_coherence
topic_num,,
32,"0.003*""boil"" + 0.002*""eggs"" + 0.002*""salt"" + 0.002*""butter"" + 0.002*""pound"" + 0.002*""sugar"" + 0.002*""pour"" + 0.002*""uncle"" + 0.002*""cream"" + 0.002*""stir""",0.868293
1,"0.004*""dat"" + 0.004*""ter"" + 0.004*""dey"" + 0.003*""ye"" + 0.003*""en"" + 0.002*""aunt"" + 0.002*""git"" + 0.002*""wid"" + 0.002*""fer"" + 0.002*""uncle""",0.832853
49,"0.003*""troops"" + 0.003*""colonel"" + 0.002*""officers"" + 0.002*""soldiers"" + 0.002*""guns"" + 0.002*""division"" + 0.002*""regiment"" + 0.002*""cavalry"" + 0.002*""officer"" + 0.002*""brigade""",0.793089
27,"0.009*""km"" + 0.006*""billion"" + 0.006*""na"" + 0.005*""million"" + 0.004*""national"" + 0.004*""military"" + 0.004*""population"" + 0.004*""islands"" + 0.004*""december"" + 0.003*""total""",0.791467
7,"0.009*""unto"" + 0.009*""thou"" + 0.009*""thy"" + 0.006*""ye"" + 0.005*""jesus"" + 0.005*""hath"" + 0.005*""christ"" + 0.004*""divine"" + 0.003*""thee"" + 0.002*""spiritual""",0.746650
22,"0.013*""thou"" + 0.012*""thy"" + 0.005*""thee"" + 0.003*""hath"" + 0.003*""ye"" + 0.002*""sir"" + 0.002*""hast"" + 0.002*""queen"" + 0.002*""unto"" + 0.001*""prince""",0.684785
33,"0.005*""la"" + 0.004*""duke"" + 0.003*""et"" + 0.003*""le"" + 0.003*""queen"" + 0.003*""prince"" + 0.002*""charles"" + 0.002*""emperor"" + 0.002*""des"" + 0.002*""france""",0.677824
28,"0.003*""per"" + 0.002*""acid"" + 0.002*""solution"" + 0.002*""oil"" + 0.002*""pressure"" + 0.002*""temperature"" + 0.002*""steam"" + 0.002*""quantity"" + 0.001*""gas"" + 0.001*""inches""",0.658910
9,"0.003*""political"" + 0.003*""british"" + 0.003*""national"" + 0.002*""german"" + 0.002*""military"" + 0.002*""congress"" + 0.002*""president"" + 0.002*""religious"" + 0.001*""constitution"" + 0.001*""france""",0.655476


In [71]:
mod_3_df, mod_3_cv_coherence, mod_3_cv_topic_coherence, mod_3_run_params =\
    analyze_model_coherence('gpu_mod_run_3', corp_lst_stream)
    
print(mod_3_cv_coherence, mod_3_run_params)
mod_3_df

2018-03-28 14:32:57,060 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model
2018-03-28 14:32:57,075 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model.expElogbeta.npy with mmap=None
2018-03-28 14:32:57,130 : INFO : setting ignored attribute dispatcher to None
2018-03-28 14:32:57,132 : INFO : setting ignored attribute state to None
2018-03-28 14:32:57,134 : INFO : setting ignored attribute id2word to None
2018-03-28 14:32:57,136 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model
2018-03-28 14:32:57,137 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model, assuming np.float64
2018-03-28 14:32:57,139 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model.state


corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_mod_run_params.txt


2018-03-28 14:32:57,633 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model.state
2018-03-28 14:32:57,635 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-03-28 14:37:52,465 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_coherence_model.cv, separately None
2018-03-28 14:37:54,012 : INFO : saved ../../outputs-git_ignored/gpu_mod_run_3/gpu_mod_run_3_coherence_model.cv
2018-03-28 14:37:54,018 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate proba

2018-03-28 14:40:05,020 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37205810 virtual)
2018-03-28 14:40:05,251 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37561483 virtual)
2018-03-28 14:40:07,023 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38210772 virtual)
2018-03-28 14:40:11,044 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38652328 virtual)
2018-03-28 14:40:12,576 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39110495 virtual)
2018-03-28 14:40:17,678 : INFO : 70 batches submitted to accumulate stats from 4480 documents (39648621 virtual)
2018-03-28 14:40:18,131 : INFO : 71 batches submitted to accumulate stats from 4544 documents (40279388 virtual)
2018-03-28 14:40:23,818 : INFO : 72 batches submitted to accumulate stats from 4608 documents (40793287 virtual)
2018-03-28 14:40:24,177 : INFO : 73 batches submitted to accumulate stats from 4672 documents (4

2018-03-28 14:42:48,070 : INFO : 138 batches submitted to accumulate stats from 8832 documents (77999717 virtual)
2018-03-28 14:42:50,540 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78530311 virtual)
2018-03-28 14:42:54,832 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79124977 virtual)
2018-03-28 14:42:56,181 : INFO : 141 batches submitted to accumulate stats from 9024 documents (79695713 virtual)
2018-03-28 14:42:56,911 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80254766 virtual)
2018-03-28 14:42:57,368 : INFO : 143 batches submitted to accumulate stats from 9152 documents (80903576 virtual)
2018-03-28 14:42:58,672 : INFO : 144 batches submitted to accumulate stats from 9216 documents (81476734 virtual)
2018-03-28 14:43:06,303 : INFO : 145 batches submitted to accumulate stats from 9280 documents (82133397 virtual)
2018-03-28 14:43:07,214 : INFO : 146 batches submitted to accumulate stats from 9344 doc

...

2018-03-28 14:47:35,586 : INFO : 281 batches submitted to accumulate stats from 17984 documents (148218137 virtual)
2018-03-28 14:47:38,444 : INFO : 282 batches submitted to accumulate stats from 18048 documents (148764608 virtual)
2018-03-28 14:47:38,679 : INFO : 283 batches submitted to accumulate stats from 18112 documents (149116468 virtual)
2018-03-28 14:47:39,010 : INFO : 284 batches submitted to accumulate stats from 18176 documents (149493413 virtual)
2018-03-28 14:47:44,089 : INFO : 285 batches submitted to accumulate stats from 18240 documents (149882584 virtual)
2018-03-28 14:47:45,789 : INFO : 286 batches submitted to accumulate stats from 18304 documents (150310921 virtual)
2018-03-28 14:47:46,804 : INFO : 287 batches submitted to accumulate stats from 18368 documents (150636123 virtual)
2018-03-28 14:47:48,951 : INFO : 288 batches submitted to accumulate stats from 18432 documents (151156589 virtual)
2018-03-28 14:47:49,855 : INFO : 289 batches submitted to accumulate sta

2018-03-28 14:50:01,939 : INFO : 352 batches submitted to accumulate stats from 22528 documents (186834449 virtual)
2018-03-28 14:50:02,501 : INFO : 353 batches submitted to accumulate stats from 22592 documents (187313847 virtual)
2018-03-28 14:50:06,596 : INFO : 354 batches submitted to accumulate stats from 22656 documents (187782848 virtual)
2018-03-28 14:50:11,168 : INFO : 355 batches submitted to accumulate stats from 22720 documents (188366342 virtual)
2018-03-28 14:50:12,261 : INFO : 356 batches submitted to accumulate stats from 22784 documents (188908503 virtual)
2018-03-28 14:50:16,227 : INFO : 357 batches submitted to accumulate stats from 22848 documents (189394598 virtual)
2018-03-28 14:50:18,598 : INFO : 358 batches submitted to accumulate stats from 22912 documents (190051860 virtual)
2018-03-28 14:50:19,563 : INFO : 359 batches submitted to accumulate stats from 22976 documents (190499622 virtual)
2018-03-28 14:50:21,170 : INFO : 360 batches submitted to accumulate sta

2018-03-28 14:52:50,669 : INFO : 423 batches submitted to accumulate stats from 27072 documents (230329083 virtual)
2018-03-28 14:52:59,966 : INFO : 424 batches submitted to accumulate stats from 27136 documents (230995513 virtual)
2018-03-28 14:53:00,333 : INFO : 425 batches submitted to accumulate stats from 27200 documents (231556984 virtual)
2018-03-28 14:53:01,594 : INFO : 426 batches submitted to accumulate stats from 27264 documents (232178205 virtual)
2018-03-28 14:53:04,851 : INFO : 427 batches submitted to accumulate stats from 27328 documents (232851488 virtual)
2018-03-28 14:53:05,966 : INFO : 428 batches submitted to accumulate stats from 27392 documents (233684381 virtual)
2018-03-28 14:53:06,455 : INFO : 429 batches submitted to accumulate stats from 27456 documents (234261498 virtual)
2018-03-28 14:53:14,734 : INFO : 430 batches submitted to accumulate stats from 27520 documents (235023215 virtual)
2018-03-28 14:53:15,116 : INFO : 431 batches submitted to accumulate sta

0.46248222267854033 {'num_topics': 30, 'cores': 41, 'chunksize': 2000, 'passes': 5, 'batch': False, 'alpha': 'symmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01, 'per_word_topic': 0.01, 'update_every': 0, 'dim_reduction_run_used': 'dr_500k'}


,topic_terms,cv_coherence
topic_num,,
8,"0.007*""km"" + 0.004*""billion"" + 0.004*""na"" + 0.004*""million"" + 0.004*""national"" + 0.003*""population"" + 0.003*""military"" + 0.003*""total"" + 0.003*""president"" + 0.003*""islands""",0.783195
2,"0.015*""thou"" + 0.013*""thy"" + 0.006*""unto"" + 0.005*""thee"" + 0.005*""hath"" + 0.004*""ye"" + 0.003*""jesus"" + 0.002*""hast"" + 0.002*""christ"" + 0.002*""divine""",0.733490
6,"0.005*""german"" + 0.004*""british"" + 0.002*""officers"" + 0.002*""troops"" + 0.002*""germans"" + 0.002*""guns"" + 0.002*""sir"" + 0.002*""military"" + 0.001*""officer"" + 0.001*""russian""",0.651828
27,"0.003*""political"" + 0.002*""national"" + 0.002*""roman"" + 0.002*""congress"" + 0.002*""president"" + 0.002*""constitution"" + 0.001*""british"" + 0.001*""military"" + 0.001*""rights"" + 0.001*""bill""",0.635148
20,"0.004*""indians"" + 0.004*""indian"" + 0.003*""fort"" + 0.002*""colonel"" + 0.002*""british"" + 0.002*""officers"" + 0.002*""camp"" + 0.002*""sir"" + 0.002*""troops"" + 0.001*""lake""",0.615753
1,"0.002*""ships"" + 0.002*""boat"" + 0.002*""island"" + 0.001*""british"" + 0.001*""sir"" + 0.001*""coast"" + 0.001*""cape"" + 0.001*""vessel"" + 0.001*""thou"" + 0.001*""natives""",0.606940
16,"0.004*""la"" + 0.003*""et"" + 0.002*""duke"" + 0.002*""charles"" + 0.002*""sir"" + 0.002*""le"" + 0.002*""prince"" + 0.002*""france"" + 0.002*""queen"" + 0.001*""henry""",0.603991
0,"0.001*""mental"" + 0.001*""science"" + 0.001*""sir"" + 0.001*""spiritual"" + 0.001*""religious"" + 0.001*""religion"" + 0.001*""literary"" + 0.001*""divine"" + 0.001*""intellectual"" + 0.001*""relation""",0.579982
15,"0.003*""british"" + 0.002*""chinese"" + 0.002*""president"" + 0.002*""military"" + 0.002*""spanish"" + 0.002*""german"" + 0.001*""congress"" + 0.001*""national"" + 0.001*""island"" + 0.001*""per""",0.536618


In [70]:
mod_4_df, mod_4_cv_coherence, mod_4_cv_topic_coherence, mod_4_run_params =\
    analyze_model_coherence('gpu_mod_run_4', corp_lst_stream)
    
print(mod_4_cv_coherence, mod_4_run_params)
mod_4_df

2018-03-28 14:11:50,246 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model
2018-03-28 14:11:50,273 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.expElogbeta.npy with mmap=None
2018-03-28 14:11:50,318 : INFO : setting ignored attribute id2word to None
2018-03-28 14:11:50,320 : INFO : setting ignored attribute state to None
2018-03-28 14:11:50,322 : INFO : setting ignored attribute dispatcher to None
2018-03-28 14:11:50,323 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model
2018-03-28 14:11:50,326 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model, assuming np.float64
2018-03-28 14:11:50,327 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.state


corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_mod_run_params.txt


2018-03-28 14:11:50,666 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.state
2018-03-28 14:11:50,669 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-03-28 14:15:59,895 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_coherence_model.cv, separately None
2018-03-28 14:16:00,959 : INFO : saved ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_coherence_model.cv
2018-03-28 14:16:00,964 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate proba

2018-03-28 14:18:18,556 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37202399 virtual)
2018-03-28 14:18:18,766 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37560385 virtual)
2018-03-28 14:18:21,126 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38196793 virtual)
2018-03-28 14:18:23,271 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38652126 virtual)
2018-03-28 14:18:26,119 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39110293 virtual)
2018-03-28 14:18:30,528 : INFO : 70 batches submitted to accumulate stats from 4480 documents (39648419 virtual)
2018-03-28 14:18:31,664 : INFO : 71 batches submitted to accumulate stats from 4544 documents (40279186 virtual)
2018-03-28 14:18:38,105 : INFO : 72 batches submitted to accumulate stats from 4608 documents (40793085 virtual)
2018-03-28 14:18:39,010 : INFO : 73 batches submitted to accumulate stats from 4672 documents (4

2018-03-28 14:21:06,516 : INFO : 138 batches submitted to accumulate stats from 8832 documents (77999515 virtual)
2018-03-28 14:21:09,660 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78530109 virtual)
2018-03-28 14:21:13,952 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79124775 virtual)
2018-03-28 14:21:14,856 : INFO : 141 batches submitted to accumulate stats from 9024 documents (79695511 virtual)
2018-03-28 14:21:15,543 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80254564 virtual)
2018-03-28 14:21:16,718 : INFO : 143 batches submitted to accumulate stats from 9152 documents (80903374 virtual)
2018-03-28 14:21:17,070 : INFO : 144 batches submitted to accumulate stats from 9216 documents (81476532 virtual)
2018-03-28 14:21:25,942 : INFO : 145 batches submitted to accumulate stats from 9280 documents (82133195 virtual)
2018-03-28 14:21:26,728 : INFO : 146 batches submitted to accumulate stats from 9344 doc

...

2018-03-28 14:26:06,252 : INFO : 281 batches submitted to accumulate stats from 17984 documents (148181663 virtual)
2018-03-28 14:26:07,558 : INFO : 282 batches submitted to accumulate stats from 18048 documents (148737749 virtual)
2018-03-28 14:26:07,795 : INFO : 283 batches submitted to accumulate stats from 18112 documents (149111128 virtual)
2018-03-28 14:26:10,220 : INFO : 284 batches submitted to accumulate stats from 18176 documents (149436808 virtual)
2018-03-28 14:26:14,099 : INFO : 285 batches submitted to accumulate stats from 18240 documents (149843533 virtual)
2018-03-28 14:26:15,896 : INFO : 286 batches submitted to accumulate stats from 18304 documents (150294597 virtual)
2018-03-28 14:26:18,107 : INFO : 287 batches submitted to accumulate stats from 18368 documents (150620000 virtual)
2018-03-28 14:26:20,399 : INFO : 288 batches submitted to accumulate stats from 18432 documents (151103828 virtual)
2018-03-28 14:26:20,638 : INFO : 289 batches submitted to accumulate sta

2018-03-28 14:28:35,578 : INFO : 352 batches submitted to accumulate stats from 22528 documents (186793282 virtual)
2018-03-28 14:28:37,302 : INFO : 353 batches submitted to accumulate stats from 22592 documents (187270262 virtual)
2018-03-28 14:28:40,856 : INFO : 354 batches submitted to accumulate stats from 22656 documents (187734468 virtual)
2018-03-28 14:28:45,627 : INFO : 355 batches submitted to accumulate stats from 22720 documents (188332002 virtual)
2018-03-28 14:28:48,052 : INFO : 356 batches submitted to accumulate stats from 22784 documents (188874893 virtual)
2018-03-28 14:28:49,866 : INFO : 357 batches submitted to accumulate stats from 22848 documents (189335232 virtual)
2018-03-28 14:28:53,002 : INFO : 358 batches submitted to accumulate stats from 22912 documents (189998909 virtual)
2018-03-28 14:28:54,104 : INFO : 359 batches submitted to accumulate stats from 22976 documents (190452972 virtual)
2018-03-28 14:28:55,093 : INFO : 360 batches submitted to accumulate sta

2018-03-28 14:31:34,818 : INFO : 423 batches submitted to accumulate stats from 27072 documents (230295821 virtual)
2018-03-28 14:31:42,525 : INFO : 424 batches submitted to accumulate stats from 27136 documents (230932605 virtual)
2018-03-28 14:31:44,629 : INFO : 425 batches submitted to accumulate stats from 27200 documents (231538568 virtual)
2018-03-28 14:31:45,061 : INFO : 426 batches submitted to accumulate stats from 27264 documents (232165409 virtual)
2018-03-28 14:31:49,938 : INFO : 427 batches submitted to accumulate stats from 27328 documents (232822306 virtual)
2018-03-28 14:31:50,589 : INFO : 428 batches submitted to accumulate stats from 27392 documents (233639502 virtual)
2018-03-28 14:31:51,057 : INFO : 429 batches submitted to accumulate stats from 27456 documents (234231618 virtual)
2018-03-28 14:31:59,119 : INFO : 430 batches submitted to accumulate stats from 27520 documents (234989616 virtual)
2018-03-28 14:31:59,679 : INFO : 431 batches submitted to accumulate sta

0.5797411828978578 {'num_topics': 20, 'cores': 9, 'chunksize': 2000, 'passes': 5, 'batch': False, 'alpha': 'symmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01, 'per_word_topic': 0.01, 'update_every': 0, 'dim_reduction_run_used': 'dr_500k'}


,topic_terms,cv_coherence
topic_num,,
18,"0.003*""salt"" + 0.003*""boil"" + 0.003*""milk"" + 0.003*""sugar"" + 0.002*""butter"" + 0.002*""pound"" + 0.002*""pour"" + 0.002*""mix"" + 0.002*""cream"" + 0.002*""boiling""",0.905420
6,"0.009*""km"" + 0.006*""billion"" + 0.006*""na"" + 0.005*""million"" + 0.004*""december"" + 0.004*""population"" + 0.004*""military"" + 0.004*""national"" + 0.004*""islands"" + 0.004*""comparison""",0.790977
9,"0.010*""la"" + 0.004*""et"" + 0.004*""le"" + 0.004*""que"" + 0.003*""en"" + 0.003*""les"" + 0.003*""des"" + 0.002*""du"" + 0.002*""el"" + 0.002*""sir""",0.755164
15,"0.016*""thou"" + 0.015*""thy"" + 0.006*""thee"" + 0.005*""unto"" + 0.004*""ye"" + 0.004*""hath"" + 0.002*""hast"" + 0.002*""shalt"" + 0.001*""thine"" + 0.001*""prince""",0.735012
7,"0.005*""british"" + 0.004*""german"" + 0.004*""troops"" + 0.003*""officers"" + 0.003*""colonel"" + 0.003*""fort"" + 0.002*""military"" + 0.002*""guns"" + 0.002*""officer"" + 0.002*""soldiers""",0.670527
17,"0.003*""indian"" + 0.003*""indians"" + 0.002*""island"" + 0.002*""lake"" + 0.002*""boat"" + 0.002*""natives"" + 0.002*""coast"" + 0.001*""cape"" + 0.001*""ice"" + 0.001*""spanish""",0.651971
10,"0.003*""political"" + 0.003*""national"" + 0.003*""president"" + 0.002*""congress"" + 0.002*""british"" + 0.002*""constitution"" + 0.002*""per"" + 0.002*""committee"" + 0.002*""rights"" + 0.002*""william""",0.647325
16,"0.003*""madame"" + 0.002*""la"" + 0.002*""monsieur"" + 0.001*""paris"" + 0.001*""prince"" + 0.001*""literary"" + 0.001*""louis"" + 0.001*""artist"" + 0.001*""pictures"" + 0.001*""italian""",0.585713
11,"0.012*""sir"" + 0.003*""duke"" + 0.002*""queen"" + 0.002*""henry"" + 0.002*""william"" + 0.002*""earl"" + 0.002*""charles"" + 0.002*""prince"" + 0.001*""mary"" + 0.001*""edward""",0.574628


In [68]:
mod_5_df, mod_5_cv_coherence, mod_5_cv_topic_coherence, mod_5_run_params =\
    analyze_model_coherence('gpu_mod_run_5', corp_lst_stream)
    
print(mod_5_cv_coherence, mod_5_run_params)
mod_5_df

2018-03-28 13:32:22,319 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model
2018-03-28 13:32:22,341 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model.expElogbeta.npy with mmap=None
2018-03-28 13:32:22,385 : INFO : setting ignored attribute state to None
2018-03-28 13:32:22,387 : INFO : setting ignored attribute dispatcher to None
2018-03-28 13:32:22,389 : INFO : setting ignored attribute id2word to None
2018-03-28 13:32:22,391 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model
2018-03-28 13:32:22,393 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model, assuming np.float64
2018-03-28 13:32:22,395 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model.state


corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_mod_run_params.txt


2018-03-28 13:32:22,737 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model.state
2018-03-28 13:32:22,739 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-03-28 13:36:25,400 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_coherence_model.cv, separately None
2018-03-28 13:36:26,430 : INFO : saved ../../outputs-git_ignored/gpu_mod_run_5/gpu_mod_run_5_coherence_model.cv
2018-03-28 13:36:26,434 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate proba

2018-03-28 13:37:53,536 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37364831 virtual)
2018-03-28 13:37:55,571 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37958048 virtual)
2018-03-28 13:37:56,953 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38470413 virtual)
2018-03-28 13:37:57,637 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38902475 virtual)
2018-03-28 13:37:57,988 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39445667 virtual)
2018-03-28 13:38:00,305 : INFO : 70 batches submitted to accumulate stats from 4480 documents (40060595 virtual)
2018-03-28 13:38:03,810 : INFO : 71 batches submitted to accumulate stats from 4544 documents (40555323 virtual)
2018-03-28 13:38:05,963 : INFO : 72 batches submitted to accumulate stats from 4608 documents (41005684 virtual)
2018-03-28 13:38:07,093 : INFO : 73 batches submitted to accumulate stats from 4672 documents (4

2018-03-28 13:39:41,218 : INFO : 138 batches submitted to accumulate stats from 8832 documents (78417619 virtual)
2018-03-28 13:39:46,404 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78994697 virtual)
2018-03-28 13:39:46,746 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79558792 virtual)
2018-03-28 13:39:47,102 : INFO : 141 batches submitted to accumulate stats from 9024 documents (80145466 virtual)
2018-03-28 13:39:47,544 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80790805 virtual)
2018-03-28 13:39:47,895 : INFO : 143 batches submitted to accumulate stats from 9152 documents (81375265 virtual)
2018-03-28 13:39:52,585 : INFO : 144 batches submitted to accumulate stats from 9216 documents (82002479 virtual)
2018-03-28 13:39:53,990 : INFO : 145 batches submitted to accumulate stats from 9280 documents (82571333 virtual)
2018-03-28 13:39:54,597 : INFO : 146 batches submitted to accumulate stats from 9344 doc

...

2018-03-28 13:42:48,477 : INFO : 281 batches submitted to accumulate stats from 17984 documents (149481424 virtual)
2018-03-28 13:42:51,457 : INFO : 282 batches submitted to accumulate stats from 18048 documents (149910142 virtual)
2018-03-28 13:42:53,386 : INFO : 283 batches submitted to accumulate stats from 18112 documents (150358759 virtual)
2018-03-28 13:42:53,920 : INFO : 284 batches submitted to accumulate stats from 18176 documents (150734201 virtual)
2018-03-28 13:42:55,084 : INFO : 285 batches submitted to accumulate stats from 18240 documents (151237547 virtual)
2018-03-28 13:42:55,789 : INFO : 286 batches submitted to accumulate stats from 18304 documents (151631040 virtual)
2018-03-28 13:42:57,491 : INFO : 287 batches submitted to accumulate stats from 18368 documents (152190415 virtual)
2018-03-28 13:42:58,841 : INFO : 288 batches submitted to accumulate stats from 18432 documents (152827682 virtual)
2018-03-28 13:42:59,141 : INFO : 289 batches submitted to accumulate sta

2018-03-28 13:44:28,884 : INFO : 352 batches submitted to accumulate stats from 22528 documents (188831347 virtual)
2018-03-28 13:44:32,208 : INFO : 353 batches submitted to accumulate stats from 22592 documents (189348287 virtual)
2018-03-28 13:44:32,842 : INFO : 354 batches submitted to accumulate stats from 22656 documents (190036782 virtual)
2018-03-28 13:44:33,077 : INFO : 355 batches submitted to accumulate stats from 22720 documents (190454738 virtual)
2018-03-28 13:44:33,975 : INFO : 356 batches submitted to accumulate stats from 22784 documents (190930460 virtual)
2018-03-28 13:44:36,426 : INFO : 357 batches submitted to accumulate stats from 22848 documents (191451002 virtual)
2018-03-28 13:44:39,754 : INFO : 358 batches submitted to accumulate stats from 22912 documents (191989085 virtual)
2018-03-28 13:44:40,403 : INFO : 359 batches submitted to accumulate stats from 22976 documents (192514694 virtual)
2018-03-28 13:44:40,791 : INFO : 360 batches submitted to accumulate sta

2018-03-28 13:46:23,539 : INFO : 423 batches submitted to accumulate stats from 27072 documents (232961540 virtual)
2018-03-28 13:46:24,268 : INFO : 424 batches submitted to accumulate stats from 27136 documents (233825565 virtual)
2018-03-28 13:46:24,830 : INFO : 425 batches submitted to accumulate stats from 27200 documents (234385444 virtual)
2018-03-28 13:46:28,866 : INFO : 426 batches submitted to accumulate stats from 27264 documents (235106798 virtual)
2018-03-28 13:46:29,290 : INFO : 427 batches submitted to accumulate stats from 27328 documents (235800197 virtual)
2018-03-28 13:46:31,714 : INFO : 428 batches submitted to accumulate stats from 27392 documents (236526516 virtual)
2018-03-28 13:46:32,494 : INFO : 429 batches submitted to accumulate stats from 27456 documents (236724840 virtual)
2018-03-28 13:46:44,897 : INFO : serializing accumulator to return to master...
2018-03-28 13:46:44,902 : INFO : accumulator serialized
2018-03-28 13:46:46,829 : INFO : serializing accumul

0.28193087808174444 {'num_topics': 20, 'cores': 9, 'chunksize': 2000, 'passes': 1, 'batch': False, 'alpha': 'symmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01, 'per_word_topic': 0.01, 'update_every': 0, 'dim_reduction_run_used': 'dr_500k'}


,topic_terms,cv_coherence
topic_num,,
8,"0.002*""sir"" + 0.001*""thy"" + 0.001*""british"" + 0.001*""political"" + 0.001*""henry"" + 0.001*""mr"" + 0.001*""thou"" + 0.001*""la"" + 0.001*""tom"" + 0.001*""national""",0.363355
10,"0.002*""sir"" + 0.002*""thy"" + 0.001*""ye"" + 0.001*""thou"" + 0.001*""william"" + 0.001*""mary"" + 0.001*""prince"" + 0.001*""hath"" + 0.001*""thee"" + 0.001*""duke""",0.343554
0,"0.007*""thou"" + 0.005*""thy"" + 0.002*""unto"" + 0.002*""ye"" + 0.002*""sir"" + 0.002*""thee"" + 0.002*""hath"" + 0.001*""indian"" + 0.001*""hast"" + 0.001*""british""",0.333089
9,"0.002*""thy"" + 0.002*""sir"" + 0.001*""thou"" + 0.001*""unto"" + 0.001*""prince"" + 0.001*""ye"" + 0.001*""charles"" + 0.001*""religious"" + 0.001*""british"" + 0.001*""german""",0.330761
12,"0.002*""sir"" + 0.001*""thy"" + 0.001*""thou"" + 0.001*""british"" + 0.001*""ye"" + 0.001*""la"" + 0.001*""prince"" + 0.001*""dick"" + 0.001*""island"" + 0.001*""henry""",0.316167
5,"0.002*""sir"" + 0.002*""thou"" + 0.001*""thy"" + 0.001*""ye"" + 0.001*""la"" + 0.001*""british"" + 0.001*""indian"" + 0.001*""william"" + 0.001*""et"" + 0.001*""national""",0.309251
16,"0.003*""sir"" + 0.001*""ye"" + 0.001*""thou"" + 0.001*""thy"" + 0.001*""british"" + 0.001*""political"" + 0.001*""la"" + 0.001*""national"" + 0.001*""military"" + 0.001*""henry""",0.299105
14,"0.003*""thou"" + 0.003*""thy"" + 0.001*""sir"" + 0.001*""national"" + 0.001*""thee"" + 0.001*""hath"" + 0.001*""william"" + 0.001*""political"" + 0.001*""madame"" + 0.001*""henry""",0.297100
18,"0.003*""thy"" + 0.002*""sir"" + 0.002*""thou"" + 0.001*""ye"" + 0.001*""unto"" + 0.001*""british"" + 0.001*""prince"" + 0.001*""william"" + 0.001*""la"" + 0.001*""political""",0.289054


In [69]:
mod_6_df, mod_6_cv_coherence, mod_6_cv_topic_coherence, mod_6_run_params =\
    analyze_model_coherence('gpu_mod_run_6', corp_lst_stream)
    
print(mod_6_cv_coherence, mod_6_run_params)
mod_6_df

2018-03-28 13:48:06,729 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model
2018-03-28 13:48:06,751 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model.expElogbeta.npy with mmap=None
2018-03-28 13:48:06,839 : INFO : setting ignored attribute dispatcher to None
2018-03-28 13:48:06,840 : INFO : setting ignored attribute state to None
2018-03-28 13:48:06,842 : INFO : setting ignored attribute id2word to None
2018-03-28 13:48:06,844 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model
2018-03-28 13:48:06,845 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model, assuming np.float64
2018-03-28 13:48:06,846 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model.state
2018-03-28 13:48:06,865 : INFO : loading sstats from ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model.sta

corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_mod_run_params.txt


2018-03-28 13:48:06,967 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model.state
2018-03-28 13:48:06,969 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-03-28 13:53:57,472 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_coherence_model.cv, separately None
2018-03-28 13:53:57,474 : INFO : storing np array 'sstats' to ../../outputs-git_ignored/gpu_mod_run_6/gpu_mod_run_6_coherence_model.cv._model.state.sstats.npy
2018-03-28 13:53:57,536 : INFO : storing np array 'expElogbeta' to ../.

2018-03-28 13:56:07,359 : INFO : 62 batches submitted to accumulate stats from 3968 documents (35707072 virtual)
2018-03-28 13:56:14,332 : INFO : 63 batches submitted to accumulate stats from 4032 documents (36242271 virtual)
2018-03-28 13:56:15,061 : INFO : 64 batches submitted to accumulate stats from 4096 documents (36696244 virtual)
2018-03-28 13:56:15,442 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37212121 virtual)
2018-03-28 13:56:17,181 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37567712 virtual)
2018-03-28 13:56:19,818 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38228887 virtual)
2018-03-28 13:56:20,123 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38666029 virtual)
2018-03-28 13:56:24,109 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39128634 virtual)
2018-03-28 13:56:29,707 : INFO : 70 batches submitted to accumulate stats from 4480 documents (3

2018-03-28 13:59:03,402 : INFO : 135 batches submitted to accumulate stats from 8640 documents (76299538 virtual)
2018-03-28 13:59:04,716 : INFO : 136 batches submitted to accumulate stats from 8704 documents (76859922 virtual)
2018-03-28 13:59:06,122 : INFO : 137 batches submitted to accumulate stats from 8768 documents (77472467 virtual)
2018-03-28 13:59:10,433 : INFO : 138 batches submitted to accumulate stats from 8832 documents (78039945 virtual)
2018-03-28 13:59:16,183 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78565069 virtual)
2018-03-28 13:59:16,957 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79185041 virtual)
2018-03-28 13:59:17,608 : INFO : 141 batches submitted to accumulate stats from 9024 documents (79725183 virtual)
2018-03-28 13:59:19,900 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80299282 virtual)
2018-03-28 13:59:20,814 : INFO : 143 batches submitted to accumulate stats from 9152 doc

...

2018-03-28 14:04:22,756 : INFO : 278 batches submitted to accumulate stats from 17792 documents (146813138 virtual)
2018-03-28 14:04:26,689 : INFO : 279 batches submitted to accumulate stats from 17856 documents (147255983 virtual)
2018-03-28 14:04:27,139 : INFO : 280 batches submitted to accumulate stats from 17920 documents (147720006 virtual)
2018-03-28 14:04:28,971 : INFO : 281 batches submitted to accumulate stats from 17984 documents (148213208 virtual)
2018-03-28 14:04:29,340 : INFO : 282 batches submitted to accumulate stats from 18048 documents (148759718 virtual)
2018-03-28 14:04:29,824 : INFO : 283 batches submitted to accumulate stats from 18112 documents (149114867 virtual)
2018-03-28 14:04:31,794 : INFO : 284 batches submitted to accumulate stats from 18176 documents (149470377 virtual)
2018-03-28 14:04:37,392 : INFO : 285 batches submitted to accumulate stats from 18240 documents (149880160 virtual)
2018-03-28 14:04:39,084 : INFO : 286 batches submitted to accumulate sta

2018-03-28 14:07:00,372 : INFO : 349 batches submitted to accumulate stats from 22336 documents (185444816 virtual)
2018-03-28 14:07:02,135 : INFO : 350 batches submitted to accumulate stats from 22400 documents (185948205 virtual)
2018-03-28 14:07:05,850 : INFO : 351 batches submitted to accumulate stats from 22464 documents (186358870 virtual)
2018-03-28 14:07:08,335 : INFO : 352 batches submitted to accumulate stats from 22528 documents (186834009 virtual)
2018-03-28 14:07:08,782 : INFO : 353 batches submitted to accumulate stats from 22592 documents (187313407 virtual)
2018-03-28 14:07:12,123 : INFO : 354 batches submitted to accumulate stats from 22656 documents (187782408 virtual)
2018-03-28 14:07:18,298 : INFO : 355 batches submitted to accumulate stats from 22720 documents (188365902 virtual)
2018-03-28 14:07:18,642 : INFO : 356 batches submitted to accumulate stats from 22784 documents (188908063 virtual)
2018-03-28 14:07:24,131 : INFO : 357 batches submitted to accumulate sta

2018-03-28 14:10:10,987 : INFO : 420 batches submitted to accumulate stats from 26880 documents (228509610 virtual)
2018-03-28 14:10:12,087 : INFO : 421 batches submitted to accumulate stats from 26944 documents (228928793 virtual)
2018-03-28 14:10:13,291 : INFO : 422 batches submitted to accumulate stats from 27008 documents (229656749 virtual)
2018-03-28 14:10:13,832 : INFO : 423 batches submitted to accumulate stats from 27072 documents (230319433 virtual)
2018-03-28 14:10:23,384 : INFO : 424 batches submitted to accumulate stats from 27136 documents (230981106 virtual)
2018-03-28 14:10:23,773 : INFO : 425 batches submitted to accumulate stats from 27200 documents (231552066 virtual)
2018-03-28 14:10:25,170 : INFO : 426 batches submitted to accumulate stats from 27264 documents (232175328 virtual)
2018-03-28 14:10:29,707 : INFO : 427 batches submitted to accumulate stats from 27328 documents (232850514 virtual)
2018-03-28 14:10:30,666 : INFO : 428 batches submitted to accumulate sta

0.4429589860842077 {'num_topics': 40, 'cores': 41, 'chunksize': 2000, 'passes': 5, 'batch': False, 'alpha': 'symmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01, 'per_word_topic': 0.01, 'update_every': 0, 'dim_reduction_run_used': 'dr_500k'}


,topic_terms,cv_coherence
topic_num,,
28,"0.002*""salt"" + 0.002*""sugar"" + 0.002*""boil"" + 0.002*""milk"" + 0.002*""eggs"" + 0.002*""butter"" + 0.002*""fruit"" + 0.002*""per"" + 0.002*""pound"" + 0.002*""pour""",0.790475
27,"0.010*""thou"" + 0.009*""thy"" + 0.009*""unto"" + 0.006*""ye"" + 0.005*""jesus"" + 0.005*""hath"" + 0.004*""christ"" + 0.003*""thee"" + 0.003*""divine"" + 0.002*""sin""",0.778404
4,"0.013*""thou"" + 0.013*""thy"" + 0.005*""thee"" + 0.003*""hath"" + 0.002*""ye"" + 0.002*""hast"" + 0.002*""sir"" + 0.001*""ere"" + 0.001*""song"" + 0.001*""doth""",0.755204
19,"0.004*""president"" + 0.003*""lincoln"" + 0.002*""congress"" + 0.002*""secretary"" + 0.002*""national"" + 0.002*""bill"" + 0.002*""political"" + 0.002*""senate"" + 0.002*""committee"" + 0.001*""vote""",0.735023
1,"0.004*""british"" + 0.004*""german"" + 0.004*""troops"" + 0.003*""officers"" + 0.002*""colonel"" + 0.002*""military"" + 0.002*""soldiers"" + 0.002*""officer"" + 0.002*""guns"" + 0.001*""sir""",0.732356
34,"0.006*""km"" + 0.005*""national"" + 0.004*""billion"" + 0.004*""million"" + 0.004*""na"" + 0.003*""military"" + 0.003*""population"" + 0.003*""president"" + 0.003*""total"" + 0.003*""economic""",0.715932
7,"0.009*""la"" + 0.004*""et"" + 0.004*""le"" + 0.003*""des"" + 0.002*""les"" + 0.002*""du"" + 0.002*""que"" + 0.002*""en"" + 0.001*""qui"" + 0.001*""e""",0.704893
0,"0.002*""science"" + 0.002*""theory"" + 0.001*""mental"" + 0.001*""conditions"" + 0.001*""religious"" + 0.001*""development"" + 0.001*""principle"" + 0.001*""scientific"" + 0.001*""relation"" + 0.001*""process""",0.614956
5,"0.008*""sir"" + 0.002*""duke"" + 0.002*""thou"" + 0.002*""thy"" + 0.002*""earl"" + 0.002*""ye"" + 0.002*""william"" + 0.002*""hath"" + 0.002*""james"" + 0.002*""thence""",0.607086


In [101]:
mod_7_df, mod_7_cv_coherence, mod_7_cv_topic_coherence, mod_7_run_params =\
    analyze_model_coherence('gpu_mod_run_7', corp_lst_stream)
    
print(mod_7_cv_coherence, mod_7_run_params)
mod_7_df

2018-03-30 10:02:35,613 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7.model
2018-03-30 10:02:35,652 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7.model.expElogbeta.npy with mmap=None
2018-03-30 10:02:35,774 : INFO : setting ignored attribute state to None
2018-03-30 10:02:35,776 : INFO : setting ignored attribute id2word to None
2018-03-30 10:02:35,778 : INFO : setting ignored attribute dispatcher to None
2018-03-30 10:02:35,780 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7.model
2018-03-30 10:02:35,782 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7.model, assuming np.float64
2018-03-30 10:02:35,784 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7.model.state
2018-03-30 10:02:35,796 : INFO : loading sstats from ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7.model.sta

corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7_mod_run_params.txt


2018-03-30 10:02:35,901 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7.model.state
2018-03-30 10:02:35,903 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-03-30 10:09:10,934 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7_coherence_model.cv, separately None
2018-03-30 10:09:10,938 : INFO : storing np array 'sstats' to ../../outputs-git_ignored/gpu_mod_run_7/gpu_mod_run_7_coherence_model.cv._model.state.sstats.npy
2018-03-30 10:09:11,012 : INFO : storing np array 'expElogbeta' to ../.

2018-03-30 10:11:53,005 : INFO : 62 batches submitted to accumulate stats from 3968 documents (35703571 virtual)
2018-03-30 10:12:02,267 : INFO : 63 batches submitted to accumulate stats from 4032 documents (36210981 virtual)
2018-03-30 10:12:02,581 : INFO : 64 batches submitted to accumulate stats from 4096 documents (36663722 virtual)
2018-03-30 10:12:03,780 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37202399 virtual)
2018-03-30 10:12:06,508 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37560385 virtual)
2018-03-30 10:12:07,040 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38196793 virtual)
2018-03-30 10:12:10,724 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38650823 virtual)
2018-03-30 10:12:15,241 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39098416 virtual)
2018-03-30 10:12:19,716 : INFO : 70 batches submitted to accumulate stats from 4480 documents (3

2018-03-30 10:15:39,325 : INFO : 135 batches submitted to accumulate stats from 8640 documents (76260036 virtual)
2018-03-30 10:15:39,634 : INFO : 136 batches submitted to accumulate stats from 8704 documents (76750590 virtual)
2018-03-30 10:15:43,742 : INFO : 137 batches submitted to accumulate stats from 8768 documents (77411895 virtual)
2018-03-30 10:15:49,962 : INFO : 138 batches submitted to accumulate stats from 8832 documents (77996318 virtual)
2018-03-30 10:15:52,364 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78525533 virtual)
2018-03-30 10:15:58,461 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79115828 virtual)
2018-03-30 10:16:00,452 : INFO : 141 batches submitted to accumulate stats from 9024 documents (79690300 virtual)
2018-03-30 10:16:01,869 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80250005 virtual)
2018-03-30 10:16:02,399 : INFO : 143 batches submitted to accumulate stats from 9152 doc

...

2018-03-30 10:22:33,170 : INFO : 278 batches submitted to accumulate stats from 17792 documents (146754108 virtual)
2018-03-30 10:22:38,114 : INFO : 279 batches submitted to accumulate stats from 17856 documents (147211070 virtual)
2018-03-30 10:22:39,871 : INFO : 280 batches submitted to accumulate stats from 17920 documents (147647228 virtual)
2018-03-30 10:22:42,195 : INFO : 281 batches submitted to accumulate stats from 17984 documents (148153900 virtual)
2018-03-30 10:22:43,785 : INFO : 282 batches submitted to accumulate stats from 18048 documents (148694802 virtual)
2018-03-30 10:22:46,899 : INFO : 283 batches submitted to accumulate stats from 18112 documents (149050261 virtual)
2018-03-30 10:22:47,748 : INFO : 284 batches submitted to accumulate stats from 18176 documents (149376358 virtual)
2018-03-30 10:22:51,966 : INFO : 285 batches submitted to accumulate stats from 18240 documents (149811692 virtual)
2018-03-30 10:22:55,954 : INFO : 286 batches submitted to accumulate sta

2018-03-30 10:25:59,936 : INFO : 349 batches submitted to accumulate stats from 22336 documents (185298764 virtual)
2018-03-30 10:26:05,493 : INFO : 350 batches submitted to accumulate stats from 22400 documents (185862184 virtual)
2018-03-30 10:26:08,430 : INFO : 351 batches submitted to accumulate stats from 22464 documents (186299645 virtual)
2018-03-30 10:26:08,708 : INFO : 352 batches submitted to accumulate stats from 22528 documents (186726615 virtual)
2018-03-30 10:26:09,687 : INFO : 353 batches submitted to accumulate stats from 22592 documents (187216486 virtual)
2018-03-30 10:26:17,068 : INFO : 354 batches submitted to accumulate stats from 22656 documents (187688512 virtual)
2018-03-30 10:26:23,191 : INFO : 355 batches submitted to accumulate stats from 22720 documents (188285782 virtual)
2018-03-30 10:26:26,318 : INFO : 356 batches submitted to accumulate stats from 22784 documents (188814619 virtual)
2018-03-30 10:26:30,800 : INFO : 357 batches submitted to accumulate sta

2018-03-30 10:30:19,346 : INFO : 420 batches submitted to accumulate stats from 26880 documents (228356021 virtual)
2018-03-30 10:30:26,181 : INFO : 421 batches submitted to accumulate stats from 26944 documents (228873006 virtual)
2018-03-30 10:30:26,834 : INFO : 422 batches submitted to accumulate stats from 27008 documents (229505025 virtual)
2018-03-30 10:30:28,030 : INFO : 423 batches submitted to accumulate stats from 27072 documents (230212977 virtual)
2018-03-30 10:30:35,816 : INFO : 424 batches submitted to accumulate stats from 27136 documents (230843416 virtual)
2018-03-30 10:30:39,016 : INFO : 425 batches submitted to accumulate stats from 27200 documents (231458188 virtual)
2018-03-30 10:30:42,882 : INFO : 426 batches submitted to accumulate stats from 27264 documents (232070188 virtual)
2018-03-30 10:30:45,445 : INFO : 427 batches submitted to accumulate stats from 27328 documents (232737865 virtual)
2018-03-30 10:30:50,375 : INFO : 428 batches submitted to accumulate sta

0.5092849769931124 {'num_topics': 50, 'cores': 41, 'chunksize': 2000, 'passes': 10, 'batch': False, 'alpha': 'symmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01, 'per_word_topic': 0.01, 'update_every': 0, 'dim_reduction_run_used': 'dr_500k'}


,topic_terms,cv_coherence
topic_num,,
24,"0.006*""boil"" + 0.006*""salt"" + 0.005*""milk"" + 0.005*""sugar"" + 0.005*""butter"" + 0.005*""pound"" + 0.004*""pour"" + 0.004*""cream"" + 0.004*""boiling"" + 0.004*""mix""",0.904741
31,"0.016*""la"" + 0.009*""et"" + 0.008*""que"" + 0.007*""le"" + 0.006*""en"" + 0.005*""les"" + 0.004*""des"" + 0.004*""el"" + 0.003*""un"" + 0.003*""du""",0.861227
30,"0.013*""km"" + 0.008*""billion"" + 0.008*""na"" + 0.007*""million"" + 0.006*""population"" + 0.006*""military"" + 0.006*""national"" + 0.005*""islands"" + 0.005*""december"" + 0.005*""total""",0.803653
15,"0.010*""german"" + 0.005*""british"" + 0.005*""troops"" + 0.004*""officers"" + 0.004*""military"" + 0.004*""germans"" + 0.003*""officer"" + 0.003*""colonel"" + 0.003*""soldiers"" + 0.003*""guns""",0.799359
0,"0.029*""thou"" + 0.025*""thy"" + 0.011*""thee"" + 0.009*""unto"" + 0.008*""hath"" + 0.007*""ye"" + 0.004*""hast"" + 0.003*""shalt"" + 0.003*""thine"" + 0.002*""doth""",0.785329
27,"0.005*""haue"" + 0.004*""hath"" + 0.003*""thou"" + 0.003*""vnto"" + 0.003*""ye"" + 0.002*""thy"" + 0.002*""doe"" + 0.002*""vpon"" + 0.002*""sir"" + 0.002*""hee""",0.770770
28,"0.008*""madame"" + 0.005*""la"" + 0.004*""monsieur"" + 0.004*""louis"" + 0.004*""prince"" + 0.003*""paris"" + 0.003*""duke"" + 0.003*""queen"" + 0.003*""france"" + 0.003*""le""",0.755706
22,"0.004*""divine"" + 0.004*""christ"" + 0.004*""jesus"" + 0.003*""spiritual"" + 0.003*""religious"" + 0.003*""religion"" + 0.003*""unto"" + 0.002*""doctrine"" + 0.002*""sin"" + 0.002*""ye""",0.724885
4,"0.031*""sir"" + 0.005*""duke"" + 0.004*""henry"" + 0.004*""william"" + 0.003*""thomas"" + 0.003*""queen"" + 0.003*""earl"" + 0.003*""charles"" + 0.002*""edward"" + 0.002*""thence""",0.691372


In [102]:
mod_8_df, mod_8_cv_coherence, mod_8_cv_topic_coherence, mod_8_run_params =\
    analyze_model_coherence('gpu_mod_run_8', corp_lst_stream)
    
print(mod_8_cv_coherence, mod_8_run_params)
mod_8_df

2018-03-30 10:33:31,876 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8.model
2018-03-30 10:33:31,895 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8.model.expElogbeta.npy with mmap=None
2018-03-30 10:33:31,936 : INFO : setting ignored attribute id2word to None
2018-03-30 10:33:31,938 : INFO : setting ignored attribute state to None
2018-03-30 10:33:31,940 : INFO : setting ignored attribute dispatcher to None
2018-03-30 10:33:31,941 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8.model
2018-03-30 10:33:31,942 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8.model, assuming np.float64
2018-03-30 10:33:31,943 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8.model.state


corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8_mod_run_params.txt


2018-03-30 10:33:32,272 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8.model.state
2018-03-30 10:33:32,273 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-03-30 10:37:31,054 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8_coherence_model.cv, separately None
2018-03-30 10:37:32,041 : INFO : saved ../../outputs-git_ignored/gpu_mod_run_8/gpu_mod_run_8_coherence_model.cv
2018-03-30 10:37:32,045 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate proba

2018-03-30 10:39:00,106 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37333432 virtual)
2018-03-30 10:39:02,004 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37908452 virtual)
2018-03-30 10:39:02,330 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38399852 virtual)
2018-03-30 10:39:02,585 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38840864 virtual)
2018-03-30 10:39:02,944 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39376045 virtual)
2018-03-30 10:39:05,284 : INFO : 70 batches submitted to accumulate stats from 4480 documents (39948034 virtual)
2018-03-30 10:39:11,499 : INFO : 71 batches submitted to accumulate stats from 4544 documents (40497502 virtual)
2018-03-30 10:39:11,800 : INFO : 72 batches submitted to accumulate stats from 4608 documents (40980992 virtual)
2018-03-30 10:39:13,081 : INFO : 73 batches submitted to accumulate stats from 4672 documents (4

2018-03-30 10:40:53,064 : INFO : 138 batches submitted to accumulate stats from 8832 documents (78292801 virtual)
2018-03-30 10:40:54,826 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78851339 virtual)
2018-03-30 10:40:55,248 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79491428 virtual)
2018-03-30 10:40:55,644 : INFO : 141 batches submitted to accumulate stats from 9024 documents (80062782 virtual)
2018-03-30 10:40:56,044 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80652532 virtual)
2018-03-30 10:40:58,479 : INFO : 143 batches submitted to accumulate stats from 9152 documents (81227973 virtual)
2018-03-30 10:41:01,744 : INFO : 144 batches submitted to accumulate stats from 9216 documents (81908097 virtual)
2018-03-30 10:41:03,473 : INFO : 145 batches submitted to accumulate stats from 9280 documents (82444581 virtual)
2018-03-30 10:41:04,529 : INFO : 146 batches submitted to accumulate stats from 9344 doc

...

2018-03-30 10:44:08,637 : INFO : 281 batches submitted to accumulate stats from 17984 documents (149197074 virtual)
2018-03-30 10:44:09,333 : INFO : 282 batches submitted to accumulate stats from 18048 documents (149591011 virtual)
2018-03-30 10:44:11,960 : INFO : 283 batches submitted to accumulate stats from 18112 documents (149988062 virtual)
2018-03-30 10:44:14,210 : INFO : 284 batches submitted to accumulate stats from 18176 documents (150432044 virtual)
2018-03-30 10:44:14,606 : INFO : 285 batches submitted to accumulate stats from 18240 documents (150807296 virtual)
2018-03-30 10:44:15,389 : INFO : 286 batches submitted to accumulate stats from 18304 documents (151326928 virtual)
2018-03-30 10:44:15,945 : INFO : 287 batches submitted to accumulate stats from 18368 documents (151692786 virtual)
2018-03-30 10:44:18,495 : INFO : 288 batches submitted to accumulate stats from 18432 documents (152245667 virtual)
2018-03-30 10:44:19,269 : INFO : 289 batches submitted to accumulate sta

2018-03-30 10:45:50,550 : INFO : 352 batches submitted to accumulate stats from 22528 documents (188328795 virtual)
2018-03-30 10:45:51,499 : INFO : 353 batches submitted to accumulate stats from 22592 documents (188859964 virtual)
2018-03-30 10:45:54,433 : INFO : 354 batches submitted to accumulate stats from 22656 documents (189353488 virtual)
2018-03-30 10:45:55,207 : INFO : 355 batches submitted to accumulate stats from 22720 documents (190073338 virtual)
2018-03-30 10:45:55,504 : INFO : 356 batches submitted to accumulate stats from 22784 documents (190475730 virtual)
2018-03-30 10:45:57,262 : INFO : 357 batches submitted to accumulate stats from 22848 documents (190959958 virtual)
2018-03-30 10:45:59,046 : INFO : 358 batches submitted to accumulate stats from 22912 documents (191486792 virtual)
2018-03-30 10:46:00,933 : INFO : 359 batches submitted to accumulate stats from 22976 documents (192019705 virtual)
2018-03-30 10:46:03,062 : INFO : 360 batches submitted to accumulate sta

2018-03-30 10:47:43,300 : INFO : 423 batches submitted to accumulate stats from 27072 documents (232331863 virtual)
2018-03-30 10:47:45,035 : INFO : 424 batches submitted to accumulate stats from 27136 documents (232942028 virtual)
2018-03-30 10:47:46,515 : INFO : 425 batches submitted to accumulate stats from 27200 documents (233783041 virtual)
2018-03-30 10:47:47,162 : INFO : 426 batches submitted to accumulate stats from 27264 documents (234379341 virtual)
2018-03-30 10:47:51,773 : INFO : 427 batches submitted to accumulate stats from 27328 documents (235136153 virtual)
2018-03-30 10:47:52,150 : INFO : 428 batches submitted to accumulate stats from 27392 documents (235743412 virtual)
2018-03-30 10:47:54,464 : INFO : 429 batches submitted to accumulate stats from 27456 documents (236517311 virtual)
2018-03-30 10:47:55,268 : INFO : 430 batches submitted to accumulate stats from 27520 documents (236766770 virtual)
2018-03-30 10:48:08,192 : INFO : serializing accumulator to return to ma

0.28979045051878705 {'num_topics': 20, 'workers': 40, 'chunksize': 2000, 'passes': 1, 'batch': False, 'alpha': 'asymmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01}


,topic_terms,cv_coherence
topic_num,,
15,"0.002*""sir"" + 0.001*""german"" + 0.001*""british"" + 0.001*""thou"" + 0.001*""thy"" + 0.001*""military"" + 0.001*""unto"" + 0.001*""la"" + 0.001*""political"" + 0.001*""prince""",0.351327
3,"0.003*""sir"" + 0.001*""thy"" + 0.001*""british"" + 0.001*""ye"" + 0.001*""thou"" + 0.001*""henry"" + 0.001*""boat"" + 0.001*""indian"" + 0.001*""william"" + 0.001*""religious""",0.340143
11,"0.002*""sir"" + 0.001*""peter"" + 0.001*""british"" + 0.001*""la"" + 0.001*""indian"" + 0.001*""henry"" + 0.001*""tom"" + 0.001*""thy"" + 0.001*""religious"" + 0.001*""military""",0.336725
12,"0.002*""thou"" + 0.002*""sir"" + 0.002*""thy"" + 0.001*""british"" + 0.001*""charles"" + 0.001*""ye"" + 0.001*""hath"" + 0.001*""duke"" + 0.001*""la"" + 0.001*""thee""",0.333624
4,"0.002*""thou"" + 0.002*""ye"" + 0.002*""unto"" + 0.002*""thy"" + 0.001*""sir"" + 0.001*""mary"" + 0.001*""william"" + 0.001*""national"" + 0.001*""la"" + 0.001*""british""",0.332182
13,"0.003*""thou"" + 0.002*""sir"" + 0.001*""thy"" + 0.001*""ye"" + 0.001*""german"" + 0.001*""british"" + 0.001*""national"" + 0.001*""thee"" + 0.001*""political"" + 0.001*""mary""",0.326869
9,"0.003*""sir"" + 0.002*""thy"" + 0.002*""thou"" + 0.001*""ye"" + 0.001*""la"" + 0.001*""william"" + 0.001*""charles"" + 0.001*""national"" + 0.001*""hath"" + 0.001*""religious""",0.322421
18,"0.002*""thou"" + 0.002*""sir"" + 0.001*""thy"" + 0.001*""prince"" + 0.001*""national"" + 0.001*""british"" + 0.001*""la"" + 0.001*""ye"" + 0.001*""military"" + 0.001*""unto""",0.314533
19,"0.002*""thou"" + 0.002*""sir"" + 0.002*""thy"" + 0.001*""political"" + 0.001*""british"" + 0.001*""national"" + 0.001*""ye"" + 0.001*""william"" + 0.001*""military"" + 0.001*""la""",0.312212


In [105]:
mod_9_df, mod_9_cv_coherence, mod_9_cv_topic_coherence, mod_9_run_params =\
    analyze_model_coherence('gpu_mod_run_9', corp_lst_stream)
    
print(mod_9_cv_coherence, mod_9_run_params)
mod_9_df

2018-03-31 17:17:57,982 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9.model
2018-03-31 17:17:58,019 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9.model.expElogbeta.npy with mmap=None
2018-03-31 17:17:58,044 : INFO : setting ignored attribute dispatcher to None
2018-03-31 17:17:58,047 : INFO : setting ignored attribute id2word to None
2018-03-31 17:17:58,048 : INFO : setting ignored attribute state to None
2018-03-31 17:17:58,050 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9.model
2018-03-31 17:17:58,052 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9.model, assuming np.float64
2018-03-31 17:17:58,054 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9.model.state


corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9_mod_run_params.txt


2018-03-31 17:17:58,362 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9.model.state
2018-03-31 17:17:58,364 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-03-31 17:30:43,424 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9_coherence_model.cv, separately None
2018-03-31 17:30:44,557 : INFO : saved ../../outputs-git_ignored/gpu_mod_run_9/gpu_mod_run_9_coherence_model.cv
2018-03-31 17:30:44,561 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate proba

2018-03-31 17:32:23,922 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37259769 virtual)
2018-03-31 17:32:24,222 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37709990 virtual)
2018-03-31 17:32:26,937 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38297253 virtual)
2018-03-31 17:32:27,462 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38736187 virtual)
2018-03-31 17:32:27,796 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39229612 virtual)
2018-03-31 17:32:32,550 : INFO : 70 batches submitted to accumulate stats from 4480 documents (39821091 virtual)
2018-03-31 17:32:34,034 : INFO : 71 batches submitted to accumulate stats from 4544 documents (40419612 virtual)
2018-03-31 17:32:38,036 : INFO : 72 batches submitted to accumulate stats from 4608 documents (40871010 virtual)
2018-03-31 17:32:39,246 : INFO : 73 batches submitted to accumulate stats from 4672 documents (4

2018-03-31 17:34:34,993 : INFO : 138 batches submitted to accumulate stats from 8832 documents (78113129 virtual)
2018-03-31 17:34:37,385 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78657148 virtual)
2018-03-31 17:34:39,096 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79242779 virtual)
2018-03-31 17:34:39,508 : INFO : 141 batches submitted to accumulate stats from 9024 documents (79862431 virtual)
2018-03-31 17:34:40,992 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80425010 virtual)
2018-03-31 17:34:41,900 : INFO : 143 batches submitted to accumulate stats from 9152 documents (81021862 virtual)
2018-03-31 17:34:47,753 : INFO : 144 batches submitted to accumulate stats from 9216 documents (81674492 virtual)
2018-03-31 17:34:50,234 : INFO : 145 batches submitted to accumulate stats from 9280 documents (82272077 virtual)
2018-03-31 17:34:50,867 : INFO : 146 batches submitted to accumulate stats from 9344 doc

...

2018-03-31 17:38:24,723 : INFO : 281 batches submitted to accumulate stats from 17984 documents (148665203 virtual)
2018-03-31 17:38:25,496 : INFO : 282 batches submitted to accumulate stats from 18048 documents (149028750 virtual)
2018-03-31 17:38:26,601 : INFO : 283 batches submitted to accumulate stats from 18112 documents (149369805 virtual)
2018-03-31 17:38:28,268 : INFO : 284 batches submitted to accumulate stats from 18176 documents (149805139 virtual)
2018-03-31 17:38:31,288 : INFO : 285 batches submitted to accumulate stats from 18240 documents (150228128 virtual)
2018-03-31 17:38:31,910 : INFO : 286 batches submitted to accumulate stats from 18304 documents (150589085 virtual)
2018-03-31 17:38:34,436 : INFO : 287 batches submitted to accumulate stats from 18368 documents (151030219 virtual)
2018-03-31 17:38:34,727 : INFO : 288 batches submitted to accumulate stats from 18432 documents (151474454 virtual)
2018-03-31 17:38:36,098 : INFO : 289 batches submitted to accumulate sta

2018-03-31 17:40:21,462 : INFO : 352 batches submitted to accumulate stats from 22528 documents (187263971 virtual)
2018-03-31 17:40:23,929 : INFO : 353 batches submitted to accumulate stats from 22592 documents (187728177 virtual)
2018-03-31 17:40:27,565 : INFO : 354 batches submitted to accumulate stats from 22656 documents (188325711 virtual)
2018-03-31 17:40:30,038 : INFO : 355 batches submitted to accumulate stats from 22720 documents (188868602 virtual)
2018-03-31 17:40:31,416 : INFO : 356 batches submitted to accumulate stats from 22784 documents (189328941 virtual)
2018-03-31 17:40:32,759 : INFO : 357 batches submitted to accumulate stats from 22848 documents (190018634 virtual)
2018-03-31 17:40:34,756 : INFO : 358 batches submitted to accumulate stats from 22912 documents (190482541 virtual)
2018-03-31 17:40:35,472 : INFO : 359 batches submitted to accumulate stats from 22976 documents (190933955 virtual)
2018-03-31 17:40:38,098 : INFO : 360 batches submitted to accumulate sta

2018-03-31 17:42:41,522 : INFO : 423 batches submitted to accumulate stats from 27072 documents (230996714 virtual)
2018-03-31 17:42:42,635 : INFO : 424 batches submitted to accumulate stats from 27136 documents (231560171 virtual)
2018-03-31 17:42:43,329 : INFO : 425 batches submitted to accumulate stats from 27200 documents (232201165 virtual)
2018-03-31 17:42:46,069 : INFO : 426 batches submitted to accumulate stats from 27264 documents (232855974 virtual)
2018-03-31 17:42:47,238 : INFO : 427 batches submitted to accumulate stats from 27328 documents (233705101 virtual)
2018-03-31 17:42:47,694 : INFO : 428 batches submitted to accumulate stats from 27392 documents (234309966 virtual)
2018-03-31 17:42:54,234 : INFO : 429 batches submitted to accumulate stats from 27456 documents (235028707 virtual)
2018-03-31 17:42:56,016 : INFO : 430 batches submitted to accumulate stats from 27520 documents (235654223 virtual)
2018-03-31 17:42:57,458 : INFO : 431 batches submitted to accumulate sta

0.29249220800747966 {'num_topics': 20, 'workers': 40, 'chunksize': 2000, 'passes': 1, 'batch': False, 'alpha': 'symmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 500, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01}


,topic_terms,cv_coherence
topic_num,,
16,"0.002*""sir"" + 0.001*""british"" + 0.001*""political"" + 0.001*""german"" + 0.001*""thy"" + 0.001*""thou"" + 0.001*""henry"" + 0.001*""duke"" + 0.001*""national"" + 0.001*""military""",0.382631
4,"0.003*""sir"" + 0.001*""la"" + 0.001*""et"" + 0.001*""political"" + 0.001*""roman"" + 0.001*""national"" + 0.001*""religious"" + 0.001*""ye"" + 0.001*""greek"" + 0.001*""william""",0.380459
7,"0.003*""thou"" + 0.002*""thy"" + 0.001*""hath"" + 0.001*""sir"" + 0.001*""duke"" + 0.001*""german"" + 0.001*""jesus"" + 0.001*""thee"" + 0.001*""dave"" + 0.001*""divine""",0.360983
19,"0.001*""colonel"" + 0.001*""exclaimed"" + 0.001*""military"" + 0.001*""british"" + 0.001*""officers"" + 0.001*""officer"" + 0.001*""thou"" + 0.001*""ye"" + 0.001*""boat"" + 0.001*""troops""",0.339236
12,"0.004*""thou"" + 0.004*""thy"" + 0.002*""thee"" + 0.001*""german"" + 0.001*""hath"" + 0.001*""unto"" + 0.001*""sir"" + 0.001*""british"" + 0.001*""national"" + 0.001*""political""",0.327064
2,"0.003*""thy"" + 0.003*""thou"" + 0.001*""ye"" + 0.001*""sir"" + 0.001*""political"" + 0.001*""thee"" + 0.001*""roman"" + 0.001*""prince"" + 0.001*""henry"" + 0.001*""national""",0.313988
14,"0.002*""uncle"" + 0.001*""sir"" + 0.001*""madame"" + 0.001*""william"" + 0.001*""x"" + 0.001*""dick"" + 0.001*""religion"" + 0.001*""national"" + 0.001*""charles"" + 0.001*""species""",0.308011
1,"0.002*""thou"" + 0.002*""sir"" + 0.001*""ye"" + 0.001*""charles"" + 0.001*""henry"" + 0.001*""island"" + 0.001*""mr"" + 0.001*""british"" + 0.001*""thy"" + 0.001*""prince""",0.305536
17,"0.001*""indian"" + 0.001*""mr"" + 0.001*""km"" + 0.001*""sir"" + 0.001*""per"" + 0.001*""indians"" + 0.001*""paul"" + 0.001*""british"" + 0.001*""northern"" + 0.001*""bishop""",0.304315


In [108]:
mod_10_df, mod_10_cv_coherence, mod_10_cv_topic_coherence, mod_10_run_params =\
    analyze_model_coherence('gpu_mod_run_10', corp_lst_stream)
    
print(mod_10_cv_coherence, mod_10_run_params)
mod_10_df

2018-04-01 15:57:57,626 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10.model
2018-04-01 15:57:57,650 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10.model.expElogbeta.npy with mmap=None
2018-04-01 15:57:57,687 : INFO : setting ignored attribute state to None
2018-04-01 15:57:57,689 : INFO : setting ignored attribute dispatcher to None
2018-04-01 15:57:57,691 : INFO : setting ignored attribute id2word to None
2018-04-01 15:57:57,692 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10.model
2018-04-01 15:57:57,694 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10.model, assuming np.float64
2018-04-01 15:57:57,695 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10.model.state


corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10_mod_run_params.txt


2018-04-01 15:57:58,166 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10.model.state
2018-04-01 15:57:58,167 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-04-01 16:02:40,564 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10_coherence_model.cv, separately None
2018-04-01 16:02:41,905 : INFO : saved ../../outputs-git_ignored/gpu_mod_run_10/gpu_mod_run_10_coherence_model.cv
2018-04-01 16:02:41,909 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estima

2018-04-01 16:04:49,646 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37219196 virtual)
2018-04-01 16:04:51,286 : INFO : 66 batches submitted to accumulate stats from 4224 documents (37585340 virtual)
2018-04-01 16:04:53,986 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38237624 virtual)
2018-04-01 16:04:54,874 : INFO : 68 batches submitted to accumulate stats from 4352 documents (38667637 virtual)
2018-04-01 16:04:57,335 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39134167 virtual)
2018-04-01 16:05:01,812 : INFO : 70 batches submitted to accumulate stats from 4480 documents (39697896 virtual)
2018-04-01 16:05:03,378 : INFO : 71 batches submitted to accumulate stats from 4544 documents (40325748 virtual)
2018-04-01 16:05:07,548 : INFO : 72 batches submitted to accumulate stats from 4608 documents (40804821 virtual)
2018-04-01 16:05:09,180 : INFO : 73 batches submitted to accumulate stats from 4672 documents (4

2018-04-01 16:07:21,685 : INFO : 138 batches submitted to accumulate stats from 8832 documents (78040112 virtual)
2018-04-01 16:07:27,106 : INFO : 139 batches submitted to accumulate stats from 8896 documents (78565236 virtual)
2018-04-01 16:07:27,649 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79185208 virtual)
2018-04-01 16:07:28,072 : INFO : 141 batches submitted to accumulate stats from 9024 documents (79725350 virtual)
2018-04-01 16:07:30,345 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80299449 virtual)
2018-04-01 16:07:31,539 : INFO : 143 batches submitted to accumulate stats from 9152 documents (80927752 virtual)
2018-04-01 16:07:32,550 : INFO : 144 batches submitted to accumulate stats from 9216 documents (81590040 virtual)
2018-04-01 16:07:40,686 : INFO : 145 batches submitted to accumulate stats from 9280 documents (82182795 virtual)
2018-04-01 16:07:41,756 : INFO : 146 batches submitted to accumulate stats from 9344 doc

...

2018-04-01 16:11:48,319 : INFO : 281 batches submitted to accumulate stats from 17984 documents (148299628 virtual)
2018-04-01 16:11:48,824 : INFO : 282 batches submitted to accumulate stats from 18048 documents (148797059 virtual)
2018-04-01 16:11:50,267 : INFO : 283 batches submitted to accumulate stats from 18112 documents (149145308 virtual)
2018-04-01 16:11:50,582 : INFO : 284 batches submitted to accumulate stats from 18176 documents (149513164 virtual)
2018-04-01 16:11:56,141 : INFO : 285 batches submitted to accumulate stats from 18240 documents (149910500 virtual)
2018-04-01 16:11:58,337 : INFO : 286 batches submitted to accumulate stats from 18304 documents (150335413 virtual)
2018-04-01 16:11:58,877 : INFO : 287 batches submitted to accumulate stats from 18368 documents (150657888 virtual)
2018-04-01 16:12:00,135 : INFO : 288 batches submitted to accumulate stats from 18432 documents (151185136 virtual)
2018-04-01 16:12:00,960 : INFO : 289 batches submitted to accumulate sta

2018-04-01 16:14:06,800 : INFO : 352 batches submitted to accumulate stats from 22528 documents (186912712 virtual)
2018-04-01 16:14:07,594 : INFO : 353 batches submitted to accumulate stats from 22592 documents (187410722 virtual)
2018-04-01 16:14:10,111 : INFO : 354 batches submitted to accumulate stats from 22656 documents (187823445 virtual)
2018-04-01 16:14:12,402 : INFO : 355 batches submitted to accumulate stats from 22720 documents (188433774 virtual)
2018-04-01 16:14:16,944 : INFO : 356 batches submitted to accumulate stats from 22784 documents (188952760 virtual)
2018-04-01 16:14:17,583 : INFO : 357 batches submitted to accumulate stats from 22848 documents (189590097 virtual)
2018-04-01 16:14:21,927 : INFO : 358 batches submitted to accumulate stats from 22912 documents (190141558 virtual)
2018-04-01 16:14:23,139 : INFO : 359 batches submitted to accumulate stats from 22976 documents (190549144 virtual)
2018-04-01 16:14:24,246 : INFO : 360 batches submitted to accumulate sta

2018-04-01 16:16:42,851 : INFO : 423 batches submitted to accumulate stats from 27072 documents (230453343 virtual)
2018-04-01 16:16:51,132 : INFO : 424 batches submitted to accumulate stats from 27136 documents (231072310 virtual)
2018-04-01 16:16:51,497 : INFO : 425 batches submitted to accumulate stats from 27200 documents (231638390 virtual)
2018-04-01 16:16:51,929 : INFO : 426 batches submitted to accumulate stats from 27264 documents (232293510 virtual)
2018-04-01 16:16:55,712 : INFO : 427 batches submitted to accumulate stats from 27328 documents (232923278 virtual)
2018-04-01 16:16:56,410 : INFO : 428 batches submitted to accumulate stats from 27392 documents (233766729 virtual)
2018-04-01 16:16:59,230 : INFO : 429 batches submitted to accumulate stats from 27456 documents (234403078 virtual)
2018-04-01 16:17:02,759 : INFO : 430 batches submitted to accumulate stats from 27520 documents (235100280 virtual)
2018-04-01 16:17:05,066 : INFO : 431 batches submitted to accumulate sta

0.43848283141856403 {'num_topics': 25, 'workers': 40, 'chunksize': 2000, 'passes': 5, 'batch': False, 'alpha': 'symmetric', 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01}


,topic_terms,cv_coherence
topic_num,,
13,"0.012*""thou"" + 0.011*""thy"" + 0.004*""thee"" + 0.004*""hath"" + 0.004*""unto"" + 0.004*""ye"" + 0.002*""christ"" + 0.002*""hast"" + 0.002*""sir"" + 0.002*""divine""",0.678462
16,"0.007*""sir"" + 0.003*""henry"" + 0.003*""william"" + 0.002*""charles"" + 0.002*""duke"" + 0.002*""la"" + 0.001*""james"" + 0.001*""thomas"" + 0.001*""earl"" + 0.001*""edward""",0.670278
6,"0.004*""national"" + 0.004*""president"" + 0.003*""km"" + 0.003*""political"" + 0.003*""military"" + 0.002*""population"" + 0.002*""british"" + 0.002*""million"" + 0.002*""congress"" + 0.002*""per""",0.668092
8,"0.004*""german"" + 0.004*""british"" + 0.002*""troops"" + 0.002*""military"" + 0.002*""sir"" + 0.002*""officers"" + 0.002*""guns"" + 0.002*""germans"" + 0.001*""colonel"" + 0.001*""soldiers""",0.628148
9,"0.002*""peter"" + 0.001*""boil"" + 0.001*""salt"" + 0.001*""butter"" + 0.001*""sugar"" + 0.001*""pound"" + 0.001*""pour"" + 0.001*""stir"" + 0.001*""cook"" + 0.001*""milk""",0.619971
2,"0.004*""la"" + 0.002*""duke"" + 0.002*""madame"" + 0.002*""le"" + 0.002*""france"" + 0.002*""troops"" + 0.002*""prince"" + 0.002*""queen"" + 0.002*""louis"" + 0.002*""et""",0.603268
5,"0.003*""unto"" + 0.003*""jesus"" + 0.002*""thou"" + 0.002*""ye"" + 0.002*""thy"" + 0.001*""divine"" + 0.001*""spiritual"" + 0.001*""prince"" + 0.001*""christ"" + 0.001*""religious""",0.593304
24,"0.002*""science"" + 0.002*""sir"" + 0.002*""mental"" + 0.001*""theory"" + 0.001*""divine"" + 0.001*""scientific"" + 0.001*""principle"" + 0.001*""spiritual"" + 0.001*""conditions"" + 0.001*""relation""",0.540486
11,"0.002*""boat"" + 0.001*""tom"" + 0.001*""exclaimed"" + 0.001*""jack"" + 0.001*""sir"" + 0.001*""uncle"" + 0.001*""indian"" + 0.001*""guess"" + 0.001*""camp"" + 0.001*""ye""",0.474567


In [109]:
mod_11_df, mod_11_cv_coherence, mod_11_cv_topic_coherence, mod_11_run_params =\
    analyze_model_coherence('gpu_mod_run_11', corp_lst_stream)
    
print(mod_11_cv_coherence, mod_11_run_params)
mod_11_df

2018-04-01 20:26:33,971 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11.model
2018-04-01 20:26:33,992 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11.model.expElogbeta.npy with mmap=None
2018-04-01 20:26:34,043 : INFO : setting ignored attribute state to None
2018-04-01 20:26:34,045 : INFO : setting ignored attribute id2word to None
2018-04-01 20:26:34,046 : INFO : setting ignored attribute dispatcher to None
2018-04-01 20:26:34,048 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11.model
2018-04-01 20:26:34,049 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11.model, assuming np.float64
2018-04-01 20:26:34,050 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11.model.state


corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11_mod_run_params.txt


2018-04-01 20:26:34,385 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11.model.state
2018-04-01 20:26:34,386 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-04-01 20:30:25,383 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11_coherence_model.cv, separately None
2018-04-01 20:30:26,419 : INFO : saved ../../outputs-git_ignored/gpu_mod_run_11/gpu_mod_run_11_coherence_model.cv
2018-04-01 20:30:26,423 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estima

2018-04-01 20:31:48,451 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37501745 virtual)
2018-04-01 20:31:48,943 : INFO : 66 batches submitted to accumulate stats from 4224 documents (38138541 virtual)
2018-04-01 20:31:49,212 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38606017 virtual)
2018-04-01 20:31:50,678 : INFO : 68 batches submitted to accumulate stats from 4352 documents (39073472 virtual)
2018-04-01 20:31:55,213 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39584587 virtual)
2018-04-01 20:31:55,666 : INFO : 70 batches submitted to accumulate stats from 4480 documents (40255963 virtual)
2018-04-01 20:31:59,360 : INFO : 71 batches submitted to accumulate stats from 4544 documents (40711913 virtual)
2018-04-01 20:31:59,619 : INFO : 72 batches submitted to accumulate stats from 4608 documents (41139871 virtual)
2018-04-01 20:32:00,780 : INFO : 73 batches submitted to accumulate stats from 4672 documents (4

2018-04-01 20:33:30,899 : INFO : 138 batches submitted to accumulate stats from 8832 documents (78595315 virtual)
2018-04-01 20:33:32,315 : INFO : 139 batches submitted to accumulate stats from 8896 documents (79207999 virtual)
2018-04-01 20:33:32,663 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79791047 virtual)
2018-04-01 20:33:35,406 : INFO : 141 batches submitted to accumulate stats from 9024 documents (80351063 virtual)
2018-04-01 20:33:35,878 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80970748 virtual)
2018-04-01 20:33:36,276 : INFO : 143 batches submitted to accumulate stats from 9152 documents (81627285 virtual)
2018-04-01 20:33:40,837 : INFO : 144 batches submitted to accumulate stats from 9216 documents (82236303 virtual)
2018-04-01 20:33:42,098 : INFO : 145 batches submitted to accumulate stats from 9280 documents (82749362 virtual)
2018-04-01 20:33:42,587 : INFO : 146 batches submitted to accumulate stats from 9344 doc

...

2018-04-01 20:36:30,742 : INFO : 280 batches submitted to accumulate stats from 17920 documents (149362973 virtual)
2018-04-01 20:36:30,989 : INFO : 281 batches submitted to accumulate stats from 17984 documents (149800501 virtual)
2018-04-01 20:36:32,958 : INFO : 282 batches submitted to accumulate stats from 18048 documents (150229984 virtual)
2018-04-01 20:36:34,862 : INFO : 283 batches submitted to accumulate stats from 18112 documents (150555363 virtual)
2018-04-01 20:36:36,030 : INFO : 284 batches submitted to accumulate stats from 18176 documents (151096569 virtual)
2018-04-01 20:36:36,703 : INFO : 285 batches submitted to accumulate stats from 18240 documents (151503926 virtual)
2018-04-01 20:36:36,928 : INFO : 286 batches submitted to accumulate stats from 18304 documents (151888178 virtual)
2018-04-01 20:36:39,615 : INFO : 287 batches submitted to accumulate stats from 18368 documents (152699978 virtual)
2018-04-01 20:36:40,980 : INFO : 288 batches submitted to accumulate sta

2018-04-01 20:38:05,916 : INFO : 351 batches submitted to accumulate stats from 22464 documents (188660201 virtual)
2018-04-01 20:38:08,306 : INFO : 352 batches submitted to accumulate stats from 22528 documents (189171271 virtual)
2018-04-01 20:38:10,401 : INFO : 353 batches submitted to accumulate stats from 22592 documents (189832277 virtual)
2018-04-01 20:38:10,677 : INFO : 354 batches submitted to accumulate stats from 22656 documents (190313817 virtual)
2018-04-01 20:38:13,329 : INFO : 355 batches submitted to accumulate stats from 22720 documents (190745352 virtual)
2018-04-01 20:38:14,241 : INFO : 356 batches submitted to accumulate stats from 22784 documents (191249723 virtual)
2018-04-01 20:38:16,942 : INFO : 357 batches submitted to accumulate stats from 22848 documents (191854113 virtual)
2018-04-01 20:38:17,744 : INFO : 358 batches submitted to accumulate stats from 22912 documents (192339658 virtual)
2018-04-01 20:38:18,112 : INFO : 359 batches submitted to accumulate sta

2018-04-01 20:40:00,181 : INFO : 422 batches submitted to accumulate stats from 27008 documents (232745734 virtual)
2018-04-01 20:40:00,807 : INFO : 423 batches submitted to accumulate stats from 27072 documents (233596218 virtual)
2018-04-01 20:40:01,277 : INFO : 424 batches submitted to accumulate stats from 27136 documents (234205967 virtual)
2018-04-01 20:40:05,829 : INFO : 425 batches submitted to accumulate stats from 27200 documents (234923313 virtual)
2018-04-01 20:40:06,481 : INFO : 426 batches submitted to accumulate stats from 27264 documents (235530876 virtual)
2018-04-01 20:40:07,652 : INFO : 427 batches submitted to accumulate stats from 27328 documents (236255734 virtual)
2018-04-01 20:40:07,952 : INFO : 428 batches submitted to accumulate stats from 27392 documents (236715731 virtual)
2018-04-01 20:40:23,650 : INFO : serializing accumulator to return to master...
2018-04-01 20:40:23,655 : INFO : accumulator serialized
2018-04-01 20:40:23,854 : INFO : serializing accumul

0.2870618084913063 {'num_topics': 20, 'workers': 40, 'chunksize': 2000, 'passes': 1, 'batch': False, 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01}


,topic_terms,cv_coherence
topic_num,,
1,"0.002*""sir"" + 0.001*""national"" + 0.001*""british"" + 0.001*""thy"" + 0.001*""president"" + 0.001*""military"" + 0.001*""william"" + 0.001*""political"" + 0.001*""la"" + 0.001*""thou""",0.390673
16,"0.001*""thou"" + 0.001*""sir"" + 0.001*""ye"" + 0.001*""british"" + 0.001*""thy"" + 0.001*""henry"" + 0.001*""duke"" + 0.001*""national"" + 0.001*""political"" + 0.001*""prince""",0.380603
15,"0.003*""thy"" + 0.003*""thou"" + 0.001*""ye"" + 0.001*""sir"" + 0.001*""unto"" + 0.001*""british"" + 0.001*""thee"" + 0.001*""mary"" + 0.001*""uncle"" + 0.001*""la""",0.362061
14,"0.001*""thou"" + 0.001*""sir"" + 0.001*""british"" + 0.001*""thy"" + 0.001*""william"" + 0.001*""prince"" + 0.001*""religious"" + 0.001*""ye"" + 0.001*""political"" + 0.001*""divine""",0.335825
18,"0.001*""sir"" + 0.001*""british"" + 0.001*""thy"" + 0.001*""thou"" + 0.001*""la"" + 0.001*""ye"" + 0.001*""german"" + 0.001*""prince"" + 0.001*""religious"" + 0.001*""indian""",0.327067
12,"0.002*""sir"" + 0.002*""thou"" + 0.001*""thy"" + 0.001*""charles"" + 0.001*""british"" + 0.001*""prince"" + 0.001*""mr"" + 0.001*""henry"" + 0.001*""national"" + 0.001*""peter""",0.319221
0,"0.002*""thy"" + 0.001*""la"" + 0.001*""thou"" + 0.001*""sir"" + 0.001*""colonel"" + 0.001*""national"" + 0.001*""military"" + 0.001*""indian"" + 0.001*""william"" + 0.001*""ye""",0.317141
3,"0.003*""thou"" + 0.002*""sir"" + 0.002*""thy"" + 0.001*""ye"" + 0.001*""william"" + 0.001*""hath"" + 0.001*""indian"" + 0.001*""thee"" + 0.001*""political"" + 0.001*""queen""",0.303281
2,"0.002*""thy"" + 0.002*""thou"" + 0.002*""sir"" + 0.001*""ye"" + 0.001*""unto"" + 0.001*""hath"" + 0.001*""thee"" + 0.001*""british"" + 0.001*""william"" + 0.001*""political""",0.298692


In [110]:
mod_12_df, mod_12_cv_coherence, mod_12_cv_topic_coherence, mod_12_run_params =\
    analyze_model_coherence('gpu_mod_run_12', corp_lst_stream)
    
print(mod_12_cv_coherence, mod_12_run_params)
mod_12_df

2018-04-01 22:59:57,544 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12.model
2018-04-01 22:59:57,564 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12.model.expElogbeta.npy with mmap=None
2018-04-01 22:59:57,594 : INFO : setting ignored attribute dispatcher to None
2018-04-01 22:59:57,595 : INFO : setting ignored attribute state to None
2018-04-01 22:59:57,597 : INFO : setting ignored attribute id2word to None
2018-04-01 22:59:57,598 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12.model
2018-04-01 22:59:57,600 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12.model, assuming np.float64
2018-04-01 22:59:57,601 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12.model.state


corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12_mod_run_params.txt


2018-04-01 22:59:57,905 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12.model.state
2018-04-01 22:59:57,906 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12.model.state, assuming np.float64
/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
2018-04-01 23:03:52,886 : INFO : saving CoherenceModel object under ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12_coherence_model.cv, separately None
2018-04-01 23:03:53,993 : INFO : saved ../../outputs-git_ignored/gpu_mod_run_12/gpu_mod_run_12_coherence_model.cv
2018-04-01 23:03:53,997 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estima

2018-04-01 23:05:18,642 : INFO : 65 batches submitted to accumulate stats from 4160 documents (37513079 virtual)
2018-04-01 23:05:19,096 : INFO : 66 batches submitted to accumulate stats from 4224 documents (38153035 virtual)
2018-04-01 23:05:19,392 : INFO : 67 batches submitted to accumulate stats from 4288 documents (38610549 virtual)
2018-04-01 23:05:21,041 : INFO : 68 batches submitted to accumulate stats from 4352 documents (39078433 virtual)
2018-04-01 23:05:25,600 : INFO : 69 batches submitted to accumulate stats from 4416 documents (39586207 virtual)
2018-04-01 23:05:26,294 : INFO : 70 batches submitted to accumulate stats from 4480 documents (40259150 virtual)
2018-04-01 23:05:29,725 : INFO : 71 batches submitted to accumulate stats from 4544 documents (40715728 virtual)
2018-04-01 23:05:29,997 : INFO : 72 batches submitted to accumulate stats from 4608 documents (41143743 virtual)
2018-04-01 23:05:31,192 : INFO : 73 batches submitted to accumulate stats from 4672 documents (4

2018-04-01 23:07:05,350 : INFO : 138 batches submitted to accumulate stats from 8832 documents (78599182 virtual)
2018-04-01 23:07:06,301 : INFO : 139 batches submitted to accumulate stats from 8896 documents (79211866 virtual)
2018-04-01 23:07:06,662 : INFO : 140 batches submitted to accumulate stats from 8960 documents (79794914 virtual)
2018-04-01 23:07:09,432 : INFO : 141 batches submitted to accumulate stats from 9024 documents (80354930 virtual)
2018-04-01 23:07:09,857 : INFO : 142 batches submitted to accumulate stats from 9088 documents (80974615 virtual)
2018-04-01 23:07:10,274 : INFO : 143 batches submitted to accumulate stats from 9152 documents (81631152 virtual)
2018-04-01 23:07:15,127 : INFO : 144 batches submitted to accumulate stats from 9216 documents (82240170 virtual)
2018-04-01 23:07:15,990 : INFO : 145 batches submitted to accumulate stats from 9280 documents (82753229 virtual)
2018-04-01 23:07:16,673 : INFO : 146 batches submitted to accumulate stats from 9344 doc

...

2018-04-01 23:10:11,114 : INFO : 281 batches submitted to accumulate stats from 17984 documents (149568638 virtual)
2018-04-01 23:10:14,425 : INFO : 282 batches submitted to accumulate stats from 18048 documents (149966584 virtual)
2018-04-01 23:10:16,791 : INFO : 283 batches submitted to accumulate stats from 18112 documents (150407896 virtual)
2018-04-01 23:10:17,033 : INFO : 284 batches submitted to accumulate stats from 18176 documents (150796506 virtual)
2018-04-01 23:10:17,338 : INFO : 285 batches submitted to accumulate stats from 18240 documents (151300481 virtual)
2018-04-01 23:10:18,543 : INFO : 286 batches submitted to accumulate stats from 18304 documents (151681810 virtual)
2018-04-01 23:10:20,869 : INFO : 287 batches submitted to accumulate stats from 18368 documents (152241634 virtual)
2018-04-01 23:10:21,682 : INFO : 288 batches submitted to accumulate stats from 18432 documents (152912491 virtual)
2018-04-01 23:10:22,922 : INFO : 289 batches submitted to accumulate sta

2018-04-01 23:11:51,204 : INFO : 352 batches submitted to accumulate stats from 22528 documents (188861958 virtual)
2018-04-01 23:11:54,014 : INFO : 353 batches submitted to accumulate stats from 22592 documents (189366018 virtual)
2018-04-01 23:11:54,946 : INFO : 354 batches submitted to accumulate stats from 22656 documents (190070256 virtual)
2018-04-01 23:11:55,303 : INFO : 355 batches submitted to accumulate stats from 22720 documents (190477094 virtual)
2018-04-01 23:11:57,111 : INFO : 356 batches submitted to accumulate stats from 22784 documents (190956727 virtual)
2018-04-01 23:11:58,962 : INFO : 357 batches submitted to accumulate stats from 22848 documents (191481935 virtual)
2018-04-01 23:12:01,355 : INFO : 358 batches submitted to accumulate stats from 22912 documents (192019950 virtual)
2018-04-01 23:12:02,651 : INFO : 359 batches submitted to accumulate stats from 22976 documents (192544998 virtual)
2018-04-01 23:12:03,050 : INFO : 360 batches submitted to accumulate sta

2018-04-01 23:13:42,986 : INFO : 423 batches submitted to accumulate stats from 27072 documents (232937621 virtual)
2018-04-01 23:13:43,601 : INFO : 424 batches submitted to accumulate stats from 27136 documents (233783300 virtual)
2018-04-01 23:13:44,023 : INFO : 425 batches submitted to accumulate stats from 27200 documents (234381499 virtual)
2018-04-01 23:13:47,872 : INFO : 426 batches submitted to accumulate stats from 27264 documents (235139909 virtual)
2018-04-01 23:13:48,446 : INFO : 427 batches submitted to accumulate stats from 27328 documents (235763548 virtual)
2018-04-01 23:13:50,445 : INFO : 428 batches submitted to accumulate stats from 27392 documents (236525469 virtual)
2018-04-01 23:13:51,746 : INFO : 429 batches submitted to accumulate stats from 27456 documents (236761829 virtual)
2018-04-01 23:14:04,123 : INFO : serializing accumulator to return to master...
2018-04-01 23:14:04,135 : INFO : accumulator serialized
2018-04-01 23:14:06,256 : INFO : serializing accumul

0.28284141422158443 {'num_topics': 20, 'workers': 40, 'chunksize': 2000, 'passes': 1, 'batch': False, 'eta': None, 'decay': 0.5, 'offset': 1.0, 'eval_every': 10, 'iterations': 50, 'gamma_threshold': 0.001, 'random_state': None, 'minimum_probability': 0.01, 'minimum_phi_value': 0.01}


,topic_terms,cv_coherence
topic_num,,
2,"0.002*""thou"" + 0.002*""la"" + 0.001*""sir"" + 0.001*""thy"" + 0.001*""british"" + 0.001*""ye"" + 0.001*""german"" + 0.001*""william"" + 0.001*""officers"" + 0.001*""et""",0.384194
10,"0.001*""thy"" + 0.001*""ye"" + 0.001*""thou"" + 0.001*""british"" + 0.001*""sir"" + 0.001*""henry"" + 0.001*""german"" + 0.001*""religious"" + 0.001*""political"" + 0.001*""national""",0.362259
11,"0.002*""thy"" + 0.002*""thou"" + 0.001*""sir"" + 0.001*""prince"" + 0.001*""ye"" + 0.001*""british"" + 0.001*""unto"" + 0.001*""la"" + 0.001*""mary"" + 0.001*""william""",0.358642
17,"0.002*""sir"" + 0.002*""thy"" + 0.001*""thou"" + 0.001*""henry"" + 0.001*""prince"" + 0.001*""british"" + 0.001*""thee"" + 0.001*""william"" + 0.001*""indian"" + 0.001*""national""",0.348457
0,"0.001*""sir"" + 0.001*""thy"" + 0.001*""british"" + 0.001*""national"" + 0.001*""aunt"" + 0.001*""political"" + 0.001*""german"" + 0.001*""thou"" + 0.001*""queen"" + 0.001*""president""",0.347151
12,"0.002*""sir"" + 0.002*""thy"" + 0.001*""thou"" + 0.001*""ye"" + 0.001*""prince"" + 0.001*""charles"" + 0.001*""political"" + 0.001*""german"" + 0.001*""military"" + 0.001*""religious""",0.332001
7,"0.002*""thou"" + 0.002*""thy"" + 0.001*""sir"" + 0.001*""la"" + 0.001*""ye"" + 0.001*""british"" + 0.001*""william"" + 0.001*""german"" + 0.001*""national"" + 0.001*""president""",0.295867
6,"0.003*""thou"" + 0.003*""sir"" + 0.002*""thy"" + 0.002*""hath"" + 0.002*""unto"" + 0.001*""ye"" + 0.001*""thee"" + 0.001*""henry"" + 0.001*""mr"" + 0.001*""german""",0.292090
8,"0.002*""thou"" + 0.001*""sir"" + 0.001*""ye"" + 0.001*""british"" + 0.001*""uncle"" + 0.001*""exclaimed"" + 0.001*""thy"" + 0.001*""military"" + 0.001*""la"" + 0.001*""national""",0.283112


In [111]:
mod_13_df, mod_13_cv_coherence, mod_13_cv_topic_coherence, mod_13_run_params =\
    analyze_model_coherence('gpu_mod_run_13', corp_lst_stream)
    
print(mod_13_cv_coherence, mod_13_run_params)
mod_13_df

2018-04-02 09:35:03,507 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_13/gpu_mod_run_13.model


corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_13/gpu_mod_run_13_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_13/gpu_mod_run_13.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_13/gpu_mod_run_13_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_13/gpu_mod_run_13.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_13/gpu_mod_run_13_mod_run_params.txt
pyLDAvis html fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_13/gpu_mod_run_13_ldavis.html
coherence model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_13/gpu_mod_run_13_coherence_model.
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_13/gpu_mod_run_13_mod_run_params.txt


FileNotFoundError: [Errno 2] No such file or directory: '../../outputs-git_ignored/gpu_mod_run_13/gpu_mod_run_13.model'

In [ ]:
#start coherence func running, mprof plot, save img - after, look at pyLDAvis, run params
#collect in a spreadsheet

In [ ]:
#for demo - look at list of titles associated with each (certain) topic(s)?

In [72]:
#set up filepaths for this model for this task
m = 'gpu_mod_run_4'
fps_test = DirFileMgr(m)
fps_test.create_all_modeling_fps(m)

#load the trained LDA model
lda = LdaModel.load(fps_test.model_fp)

2018-03-29 11:31:05,510 : INFO : loading LdaModel object from ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model
2018-03-29 11:31:05,552 : INFO : loading expElogbeta from ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.expElogbeta.npy with mmap=None
2018-03-29 11:31:05,605 : INFO : setting ignored attribute id2word to None
2018-03-29 11:31:05,607 : INFO : setting ignored attribute state to None
2018-03-29 11:31:05,608 : INFO : setting ignored attribute dispatcher to None
2018-03-29 11:31:05,610 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model
2018-03-29 11:31:05,612 : INFO : dtype was not set in saved LdaMulticore file ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model, assuming np.float64
2018-03-29 11:31:05,614 : INFO : loading LdaState object from ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.state


corpus lst fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_lst.txt
dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.dict
counts dictionary fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_json.txt
model fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model
modeling run parameters fp is assigned as  ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4_mod_run_params.txt


2018-03-29 11:31:05,947 : INFO : loaded ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.state
2018-03-29 11:31:05,955 : INFO : dtype was not set in saved LdaState file ../../outputs-git_ignored/gpu_mod_run_4/gpu_mod_run_4.model.state, assuming np.float64


In [74]:
lda.alpha

array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])

In [107]:
lda.alpha.shape

(20,)

In [75]:
lda.batch

False

In [80]:
lda.bound()

TypeError: bound() missing 1 required positional argument: 'corpus'

In [77]:
lda.chunksize

2000

In [78]:
lda.decay

0.5

In [79]:
lda.diff

<bound method LdaModel.diff of <gensim.models.ldamulticore.LdaMulticore object at 0x146725be0>>

In [81]:
lda.eta

array([0.05, 0.05, 0.05, ..., 0.05, 0.05, 0.05])

In [82]:
lda.eval_every

10

In [83]:
lda.gamma_threshold

0.001

In [84]:
lda.iterations

50

In [85]:
lda.minimum_phi_value

0.01

In [86]:
lda.minimum_probability

0.01

In [87]:
lda.num_terms

262826

In [88]:
lda.num_topics

20

In [89]:
lda.num_updates

28492

In [90]:
lda.numworkers

1

In [91]:
lda.offset

1.0

In [92]:
lda.passes

5

In [95]:
lda.random_state

In [98]:
lda.state

In [100]:
lda.update_every

1

In [99]:
lda.workers

8

In [ ]:
lda.inference

In [16]:
cm1_umass = CoherenceModel(model=lda1, \
                    corpus=serial_corp, \
                    dictionary=d, \
                    coherence='u_mass')

cm1_umass.save(fps1.coherence_model_fp + 'umass')

cm1_umass.get_coherence()

cm1_umass.get_coherence_per_topic()

In [10]:
cm1_uci = CoherenceModel(model=lda1,
                    texts=corp_lst_stream, \
                    coherence='c_uci')

cm1_uci_coherence = cm1_uci.get_coherence())
cm1_uci_topics_coherence = cm1_uci.get_coherence_per_topic())

cm1_uci.save(fps1.coherence_model_fp + 'uci')